### Package imports

In [1]:
from os import listdir, getcwd, rename, makedirs
from os.path import isfile, join, isdir, exists
import pandas as pd
import logging
from PIL import Image
from tqdm import tqdm

### Common functions

In [2]:
def get_dirs(path):
    return [f for f in listdir(path) if isdir(join(path, f))]

def get_files(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

def read_metadata_without_fields(path):
    metadata = pd.read_csv(path, sep='\t', na_values="", low_memory=False)
    if 'Field' in metadata.columns:
        metadata = metadata.loc[metadata.Field.str.startswith('#') != True,].reset_index(drop=True)
        metadata = metadata.drop('Field', axis=1)
    return metadata

def read_metadata_with_fields(path):
    metadata = pd.read_csv(path, sep='\t', na_values="", low_memory=False)
    return metadata


def sort_case_insensitive(sort_list):
    return sorted(sort_list, key=str.casefold)

def read_excel_sheets_merged(path):
    return pd.concat(pd.read_excel(path, sheet_name=None, engine='openpyxl').values(), ignore_index=True)
    

### Common env

In [3]:
start_dir = getcwd()
home = "/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/"

## Generate XLSX

In [6]:
def generate_xlsx(path, provider):
    file_list = get_files(path)
    new_path = path.replace('/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/', '/Users/tushar/CancerModels/pdxfinder-data/data/data-submission/')
    if not exists(new_path):
        makedirs(new_path)
    files_to_extract = [f for f in file_list if f.endswith('.tsv') and f.__contains__(provider+'_metadata-')]
    if len(files_to_extract)>0:
        print("writing excel for: "+ provider)
        with pd.ExcelWriter(join(new_path, provider+"_metadata.xlsx")) as writer:
            for f in files_to_extract:
                print("\tSheet added: " + f)
                metadata = read_metadata_with_fields(join(path, f))
                sheetname = f.replace(provider+"_metadata-", "").replace(".tsv", "")
                metadata.to_excel(writer, sheet_name=sheetname, index=False)
                
    else:
        print("No .xlsx file found.")

In [9]:
for provider in sorted(get_dirs(home)): ## get_dirs will get the provider dirs in updog
    #print("Working on provider: "+provider)
    generate_xlsx(join(home, provider), provider) ## File rename: pdx_models to pdx_model using the provider path

writing excel for: BROD
	Sheet added: BROD_metadata-patient_sample.tsv
	Sheet added: BROD_metadata-model_validation.tsv
	Sheet added: BROD_metadata-cell_model.tsv
	Sheet added: BROD_metadata-patient.tsv
	Sheet added: BROD_metadata-sharing.tsv
writing excel for: CCIA
	Sheet added: CCIA_metadata-model_validation.tsv
	Sheet added: CCIA_metadata-pdx_model.tsv
	Sheet added: CCIA_metadata-patient_sample.tsv
	Sheet added: CCIA_metadata-sharing.tsv
	Sheet added: CCIA_metadata-patient.tsv
writing excel for: CHOP
	Sheet added: CHOP_metadata-patient_sample.tsv
	Sheet added: CHOP_metadata-pdx_model.tsv
	Sheet added: CHOP_metadata-model_validation.tsv
	Sheet added: CHOP_metadata-sharing.tsv
	Sheet added: CHOP_metadata-patient.tsv
writing excel for: CMP
	Sheet added: CMP_metadata-patient_sample.tsv
	Sheet added: CMP_metadata-sharing.tsv
	Sheet added: CMP_metadata-patient.tsv
	Sheet added: CMP_metadata-model_validation.tsv
	Sheet added: CMP_metadata-cell_model.tsv
writing excel for: CRL
	Sheet added:

## Collection site typo


In [24]:
def get_collection_site(path, provider, cs_df):
    file_list = get_files(path)
    files_to_extract = [f for f in file_list if f.endswith('_metadata-patient_sample.tsv')]
    if len(files_to_extract)>0:
        #print("Reading Patient sample for: "+ provider)
        for f in files_to_extract:
            metadata = read_metadata_without_fields(join(path, f))
            metadata['provider'] = provider
            cs_df = pd.concat([cs_df, metadata[['collection_site', 'provider']]]).reset_index(drop=True)
    else:
        print("No file found.")
    return cs_df

In [58]:
PS_collection_site = pd.DataFrame()
for provider in sorted(get_dirs(home)): ## get_dirs will get the provider dirs in updog
    PS_collection_site = get_collection_site(join(home, provider), provider, PS_collection_site) ## File rename: pdx_models to pdx_model using the provider path
Unique_CS = PS_collection_site.sort_values(by=['collection_site'], key=lambda col: col.str.lower()).drop_duplicates(subset=['provider', 'collection_site'], keep='first').reset_index(drop=True)

In [59]:
sort_case_insensitive(list(PS_collection_site.collection_site.unique()))

['4th ventricle',
 'Abdomen',
 'Abdomen Ascites',
 'Abdominal',
 'Abdominal Cavity',
 'Abdominal Mass',
 'Abdominal Wall',
 'Abdominopelvic Cavity',
 'Adrenal',
 'Adrenal Cortex',
 'Adrenal Gland',
 'Adrenal Mass',
 'Adrenal Resection',
 'Adrenal Tissue',
 'Alveolus',
 'Ampulla',
 'Ampulla of Vater',
 'Anal',
 'Anus',
 'Aortocaval',
 'Arm',
 'Ascending Colon',
 'Ascites',
 'Ascites fluid',
 'Axilla',
 'Axillary Lymph Node',
 'Back',
 'Back Mass',
 'Back soft tissue',
 'Base Of Tongue',
 'Bilateral',
 'Bile Duct',
 'Biliary Tract',
 'Bladder',
 'Bladder Dome',
 'Blood',
 'Bone',
 'Bone (Left Proximal Humerus)',
 'Bone (right humerus)',
 'Bone Marrow',
 'Bowel',
 'Brachial Muscle',
 'Brain',
 'Brain (Right Cerebellum, Right Parietal)',
 'Brain Stem',
 'Breast',
 'Bronchus',
 'Buccal Mucosa',
 'Buttock',
 'Caecum',
 'Calf',
 'Cecum',
 'Cerebellar Tentorium',
 'Cerebellum',
 'Cerebral hemisphere',
 'Cerebrospinal fluid ',
 'Cerebrum',
 'Cervical endometrium',
 'Cervical Lymph Node',
 'Cerv

In [56]:
list(Unique_CS.collection_site.unique())

['4th ventricle',
 'Abdomen',
 'Abdomen Ascites',
 'Abdominal',
 'Abdominal Cavity',
 'Abdominal Mass',
 'Abdominal Wall',
 'Abdominopelvic Cavity',
 'Adrenal',
 'Adrenal Cortex',
 'Adrenal Gland',
 'Adrenal Mass',
 'Adrenal Resection',
 'Adrenal Tissue',
 'Alveolus',
 'Ampulla',
 'Ampulla of Vater',
 'Anal',
 'Anus',
 'Aortocaval',
 'Arm',
 'Ascending Colon',
 'Ascites',
 'Ascites ',
 'Ascites fluid',
 'Axilla',
 'Axillary Lymph Node',
 'Back',
 'Back Mass',
 'Back soft tissue',
 'Base Of Tongue',
 'Bilateral',
 'Bile Duct',
 'Biliary Tract',
 'Bladder',
 'Bladder Dome',
 'Blood',
 'Bone',
 'Bone (Left Proximal Humerus)',
 'Bone (right humerus)',
 'Bone Marrow',
 'Bowel',
 'Brachial Muscle',
 'Brain',
 'Brain (Right Cerebellum, Right Parietal)',
 'Brain Stem',
 'Breast',
 'Bronchus',
 'Buccal Mucosa',
 'Buttock',
 'Caecum',
 'Calf',
 'Cecum',
 'Cerebellar Tentorium',
 'Cerebellum',
 'Cerebral hemisphere',
 'Cerebrospinal fluid ',
 'Cerebrum',
 'Cervical endometrium',
 'Cervical Lymph 

## Data type STRING to Number

In [32]:
def change_data_type(path, provider, mol_type):
    cols = {"cna": ["log10r_cna", "log2r_cna"],
            "mut": ["read_depth"], ## "allele_frequency" and "seq_start_position: cant be set as 0
            "expression": ["z_score", "rnaseq_tpm", "rnaseq_fpkm", "rnaseq_count",
                           "affy_hgea_expression_value", "illumina_hgea_expression_value"]} ## "rnaseq_coverage" cant be set as 0

    file_list = get_files(path)
    files_to_extract = [f for f in file_list if f.endswith('.tsv')]
    fillcols = cols[mol_type]
    if len(files_to_extract)>0:
        print("Reading Patient sample for: "+ provider)
        for f in files_to_extract:
            ## Implement some checks for expression columns
            file_path = join(path, f)
            metadata = read_metadata_without_fields(file_path)
            metadata[fillcols] = metadata[fillcols].fillna(0)
            metadata.to_csv(file_path, sep='\t', index=False)
    else:
        print(provider + ": No file found.")

In [33]:
mol_types = ["cna", "mut", "expression"]
for mol_type in mol_types:
    for provider in sorted(get_dirs(home)): ## get_dirs will get the provider dirs in updog
        change_data_type(join(home, provider, mol_type), provider, mol_type) ## File rename: pdx_models to pdx_model using the

BROD: No file found.
Reading Patient sample for: CCIA
Reading Patient sample for: CHOP
Reading Patient sample for: CMP
Reading Patient sample for: CRL
CSHL: No file found.
Reading Patient sample for: CUIMC
Curie-BC: No file found.
Reading Patient sample for: Curie-LC
Curie-OC: No file found.
DFCI-CPDM: No file found.
Reading Patient sample for: GCCRI
HCI-BCM: No file found.
HKU: No file found.
Reading Patient sample for: IRCC-CRC
Reading Patient sample for: IRCC-GC
Reading Patient sample for: JAX
Reading Patient sample for: LIH
Reading Patient sample for: LurieChildrens
MDAnderson: No file found.
Reading Patient sample for: MDAnderson-CCH
Reading Patient sample for: NKI
PDMR: No file found.
Reading Patient sample for: PMLB
PMLB-Organoid: No file found.
SANG: No file found.
SJCRH: No file found.
TRACE: No file found.
UCD: No file found.
UMCG: No file found.
Reading Patient sample for: UOC-BC
UOM-BC: No file found.
VHIO-BC: No file found.
VHIO-CRC: No file found.
VHIO-PC: No file found.


1. BROD: 42
2. CCIA: 1
3. CHOP: 1
4. CMP: 2
5. CRL: 1
6. CSHL: 38
7. CUIMC: 2
8. Curie-BC: 5
9. Curie-LC: 2
10. GCCRI: 1
11. IRCC-CRC: 1
12. IRCC-GC: 1
13. JAX: 354
14. LIH: 2
15. LurieChildrens: 1
16. MDAnderson-CCH: 1
17. NKI: 7
18. PDMR: 2
19. PMLB: 3
20. TRACE: 2
21. UOC-BC: 2
22. UOM-BC: 1
23. VHIO-CRC: 2
49.375


## JAX Histology Images
### Generate file list

In [55]:
JAX_Image_file_path = "/Users/tushar/CancerModels/submission/JAX - BIA/JAXPDXHistologyImageDetails.xlsx"
JAX_BIA_fileList_path = "/Users/tushar/CancerModels/submission/JAX - BIA/JAX_PDX_BIA_fileList.xlsx"
input_path = "/Users/tushar/CancerModels/submission/JAX - BIA/uploads"
output_path = "/Users/tushar/CancerModels/submission/JAX - BIA/submission"

In [45]:
def process_JAX_histology_excel(excel_in):
    JAX_BIA_file = pd.read_excel(excel_in).sort_values("model_id").reset_index(drop=True)
    JAX_BIA_file.loc[JAX_BIA_file.description.str.contains("PDX"), "sample_type"] = "pdx"
    JAX_BIA_file.loc[JAX_BIA_file.description.str.contains("Patient"), "sample_type"] = "patient"
    JAX_BIA_file['passage'] = JAX_BIA_file['description'].str[:2]
    JAX_BIA_file.loc[JAX_BIA_file.sample_type == 'patient', "passage"] = "-"
    JAX_BIA_file = JAX_BIA_file[JAX_BIA_file.sample_type == "pdx"]
    return JAX_BIA_file

def convert_JPEG_to_TIFF(in_file, out_file):
    im = Image.open(in_file)
    im.save(out_file, 'TIFF')


def generate_TIFF_and_fileList(dataframe, in_path, out_path, df_out_file):
    if not exists(out_path):
        makedirs(out_path)
    dataframe['Files'] = 'submission/'+ dataframe['model_id'] +"/"+ dataframe['file'].str.replace(".jpg", ".tiff")
    unique_models = dataframe.model_id.unique()
    for i in tqdm(range(0, len(unique_models)), desc ="Processing model images: "):
        model = unique_models[i]
        new_path = join(out_path, model)
        if not exists(new_path):
            makedirs(new_path)
        subset = dataframe[dataframe.model_id == model]
        files = list(subset.file)
        for file in files:
            out_file = file.replace(".jpg", ".tiff")
            convert_JPEG_to_TIFF(join(in_path, file), join(new_path, out_file))
    dataframe.to_excel(df_out_file, index=False)

def organise_data_for_BIA_submission(excel_in, excel_out, file_in, file_out):
    dataframe = process_JAX_histology_excel(excel_in)
    generate_TIFF_and_fileList(dataframe, file_in, file_out, excel_out)


In [52]:
organise_data_for_BIA_submission(JAX_Image_file_path, JAX_BIA_fileList_path, input_path, output_path)

In [53]:
JAX_BIA_file['file'].str.replace(".jpg", ".tiff")

Processing model images: 100%|██████████| 433/433 [00:56<00:00,  7.73it/s]


## LIH Gene expression data

In [18]:
path = "/Users/tushar/CancerModels/submission/LIH/NORLUX_PDOX_gene_Expression/"
files = [f for f in get_files(path) if f.endswith('.xlsx')]
mol_sample = pd.read_csv("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/LIH/LIH_molecular_metadata-sample.tsv", sep='\t')

In [21]:
for i in tqdm(range(0, len(files)), desc ="Processing gene expression data: "):
    f = files[i]
    out_data = pd.read_csv("/Users/tushar/CancerModels/pdxfinder-data/template/active_templates/expression/expression_template-sheet.tsv", sep="\t")
    model = pd.read_excel(join(path, f), sheet_name=None, engine='openpyxl')
    model = pd.concat(model.values(), ignore_index=True)
    out_data[out_data.columns] = model[out_data.columns]
    out_data["platform_id"] = "expression_NovaSeq_6000"
    out_data.drop(['platform'], axis=1).to_csv( join(path, "LIH_expression_" + f.replace(".xlsx", ".tsv")), sep="\t", index=False)
    #print(model.iloc[0, ["model_id", "sample_id", "sample_origin", "passage", "host_strain_nomenclature"]])

Processing gene expression data: 100%|██████████| 28/28 [09:43<00:00, 20.84s/it]


In [102]:
import math

# Curie BC CNA data

In [112]:
cna_data_path = '/Users/tushar/CancerModels/submission/Curie/mol'
curie_bc_ps = '/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/Curie-BC/Curie-BC_metadata-patient_sample.tsv'
curie_bc_mms = '/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/Curie-BC/Curie-BC_molecular_metadata-sample.tsv'
curie_bc_mms = read_metadata_with_fields(curie_bc_mms)
platform = {"cytoscan array": "cna_cytoscan_array", "SNP6 array": "cna_snp6"}
files = [[platform[d], f.replace(' Cytoscan.xls', '').replace(' SNP6.xls', ''), join(cna_data_path, d, f)] for d in get_dirs(cna_data_path) for f in get_files(join(cna_data_path, d)) if f.__contains__('xls')]

mol_sample = '/Users/tushar/CancerModels/submission/Curie/mol/cna_template_CGH_CN.xlsx'
mol_sample = read_excel_sheets_merged(mol_sample)
mol_sample = mol_sample[['model_id', 'sample_id', 'sample_origin', 'passage', 'host_strain_nomenclature', 'platform']] 
mol_sample['platform'] = mol_sample['platform'].str.replace(' HD', '').replace(platform).str.lower().replace(platform)
mol_sample = mol_sample[mol_sample.platform.isin(platform.values())].drop_duplicates()
mol_sample = mol_sample.merge(pd.DataFrame(files, columns=['platform', 'sample_id', 'filename']), on='sample_id', how='right')
mol_sample['sample_origin'] = 'xenograft'
mol_sample['passage'] = mol_sample['passage'].fillna(0).astype(int)
mol_sample['host_strain_nomenclature'] = mol_sample['host_strain_nomenclature'].fillna('Crl:NU(Ico)-Foxn1nu').str.replace('Crl:NU(Ico)-Foxn1nu', 'Crl:NU(Ico)-Foxn1<sup>nu</sup>')
mol_sample['platform_id'] = mol_sample['platform_y']
mol_sample = mol_sample.rename(columns={'sample_id': 'model_id', 'model_id': 'sample_id'}).drop(['platform_y', 'platform_x'], axis=1)
mol_sample.loc[mol_sample['model_id'] == 'HBCx-176', 'sample_id'] = 'BC1254'
mol_sample.loc[mol_sample['model_id'] == 'HBCx-8', 'sample_id'] = 'BC138'

In [109]:
cna_template = pd.read_csv("/Users/tushar/CancerModels/pdxfinder-data/template/active_templates/cna/cna_template-sheet.tsv", sep='\t')
mapper = {'chr': 'chromosome', 'chrStart': 'seq_start_position', 'chrEnd': 'seq_end_position', 'entrezid': 'ncbi_gene_id', 'Log2Ratio': 'log2r_cna'}
for f in files:
    pid, mid, fp = f
    print(fp)
    sid = mol_sample.loc[mol_sample['model_id'] == mid, 'sample_id'].reset_index(drop=True)[0]
    if sid != None:
        cna = pd.read_excel(fp, engine='xlrd')
        cna['sample_id'] = sid
        cna['platform_id'] = pid
        cna.rename(columns=mapper, inplace=True)
        cna['fold_change'] = 2**cna['log2r_cna']
        cna['log10r_cna'] = cna['log2r_cna'] / math.log2(10)
        for c in cna_template.columns:
            if c not in cna.columns:
                cna[c] = ''
        cna = cna[cna_template.columns]
        out_file = f'Curie-BC_cna_{mid}.tsv'
        cna.to_csv(join('/Users/tushar/CancerModels/submission/Curie/cna', out_file), sep='\t', index=False)
    else:
        print('No SID')

/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-176 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-124 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-131 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-137 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-106 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-92 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-118 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-139 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-149 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-180 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBCx-151 Cytoscan.xls
/Users/tushar/CancerModels/submission/Curie/mol/cytoscan array/HBC

In [114]:
for c in curie_bc_mms.columns:
    if c not in mol_sample.columns:
        mol_sample[c] = ''
final_mms = pd.concat([curie_bc_mms, mol_sample[curie_bc_mms.columns]]).reset_index(drop=True)
final_mms

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,desirable,desirable,essential
4,NaN,HBCx-3,HBCx-3_S,xenograft,1,NaN,Crl:NU(Ico)-Foxn1<sup>nu</sup>,inter-scapular,NaN,cytogenetics_immunohistochemistry
...,...,...,...,...,...,...,...,...,...,...
70,,HBCx-2,BC51,xenograft,10,,Crl:NU(Ico)-Foxn1<sup>nu</sup>,,,cna_snp6
71,,HBCx-8,BC138,xenograft,0,,Crl:NU(Ico)-Foxn1<sup>nu</sup>,,,cna_snp6
72,,HBCx-39,BC408,xenograft,7,,Crl:NU(Ico)-Foxn1<sup>nu</sup>,,,cna_snp6
73,,HBCx-30,BC291,xenograft,8,,Crl:NU(Ico)-Foxn1<sup>nu</sup>,,,cna_snp6


In [117]:
mut = read_metadata_with_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/Curie-BC/mut/Curie-BC_mut.tsv")



['BC51',
 'BC138',
 'BC297',
 'BC408',
 'BC965',
 'BC1006',
 'BC1060',
 'BC1111',
 'BC1115',
 'BC1117',
 'BC1128',
 'BC1145',
 'BC1151',
 'BC1153',
 'BC1254']

In [122]:
set(mol_sample['sample_id'].unique()) - set(mut['sample_id'].unique()) 

{'BC1080', 'BC1101', 'BC1270', 'BC291'}

In [126]:
set(mut['platform_id'].unique())

{'mutation_NGS_ESOPE'}

In [128]:
mut_mms = mol_sample[mol_sample['sample_id'].isin(mut['sample_id'].unique())]
mut_mms['platform_id'] = 'mutation_NGS_ESOPE'
final_mms = pd.concat([final_mms, mut_mms[curie_bc_mms.columns]]).reset_index(drop=True)

/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_81977/3867220052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mut_mms['platform_id'] = 'mutation_NGS_ESOPE'


In [130]:
final_mms.to_csv('/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/Curie-BC/Curie-BC_molecular_metadata-sample.tsv', sep='\t', index=False)

# PIVOT mol data cleanup

In [7]:
PIVOT = ["CCIA","CHOP", "GCCRI", "LurieChildrens", "MDAnderson-CCH"]

In [8]:
def float_2_int(col):
    return [int(x) if x!= '' else x for x in col.fillna('')]
def convert_amino_acid_position(value):
    if isinstance(value, str):
        # Split the string by non-numeric characters and take the first part
        position_str = value[1:-1]  
        if position_str.__contains__('-'):
            return value[0] + str(position_str) + value[-1]
        return value[0] + str(int(float(position_str))) + value[-1]
    # Return NaN for non-matching or NaN values
    return ''

def drop_dec(path, dt):
    for file in get_files(path):
        fp = join(path, file)
        data = pd.read_csv(fp, sep='\t')
        if dt == 'cna':
          data.strand = data.strand.fillna(1.0).astype(int)
          data.seq_start_position = float_2_int(data.seq_start_position)
          data.seq_end_position = float_2_int(data.seq_end_position)
          data.ncbi_gene_id = float_2_int(data.ncbi_gene_id)
        elif dt == 'expression':
            data.ensembl_gene_id = float_2_int(data.ensembl_gene_id)
        elif dt == 'mut':
            data['amino_acid_change'] = data['amino_acid_change'].apply(convert_amino_acid_position)
        data = data.iloc[:, 1:]
        data.to_csv(fp, sep='\t', index=False)
def drop_decimal(provider):
    dirs = get_dirs(provider)
    dt = ['cna', 'expression', 'mut']
    for d in dt:
        if d in dirs:
            new_path = join(provider, d)
            drop_dec(new_path, d)    

In [9]:
for i in tqdm(range(0, len(PIVOT)), desc ="Processing PIVOT data: "):
    provider = PIVOT[i]
    drop_decimal(join(home, provider))

Processing PIVOT data: 100%|██████████| 5/5 [03:29<00:00, 41.87s/it]


In [10]:
PDMR_path = join(home, "PDMR/expression/")

In [12]:
for f in get_files(PDMR_path):
    data = read_metadata_with_fields(join(PDMR_path, f))
    data.drop("platform", axis=1, inplace=True)
    data.to_csv(join(PDMR_path, f), index=False, sep='\t')

In [11]:
missing_ids = "/Users/tushar/Downloads/model_molecular_metadata_202310301437.csv"
missing_ids = pd.read_csv(missing_ids).sort_values("data_source").reset_index(drop=True)

In [36]:
missing_ids

,model_id,data_source,source,sample_id,xenograft_passage,raw_data_url,data_type,platform_name,data_exists,data_restricted,molecular_characterization_id,external_db_links
0,CRL-3044,CRL,xenograft,A4632_(GenomeWideSNP_6),2.0,A4632_(GenomeWideSNP_6).CEL,copy number alteration,Affymetrix Genome-Wide Human SNP Array 6.0,False,True,1726576853006,[]
1,CRL-3039,CRL,xenograft,A6297_(GenomeWideSNP_6),0.0,A6297_(GenomeWideSNP_6).CEL,copy number alteration,Affymetrix Genome-Wide Human SNP Array 6.0,False,True,3281355014163,[]
2,CRL-3040,CRL,xenograft,A8251_(GenomeWideSNP_6),0.0,A8251_(GenomeWideSNP_6).CEL,copy number alteration,Affymetrix Genome-Wide Human SNP Array 6.0,False,True,2972117368850,[]
3,CRL-3036,CRL,xenograft,A9492_(GenomeWideSNP_6),1.0,A9492_(GenomeWideSNP_6).CEL,copy number alteration,Affymetrix Genome-Wide Human SNP Array 6.0,False,True,3083786518542,[]
4,CRL-3039,CRL,xenograft,A8445_(GenomeWideSNP_6),2.0,A8445_(GenomeWideSNP_6).CEL,copy number alteration,Affymetrix Genome-Wide Human SNP Array 6.0,False,True,3118146256916,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
240,834989-109-R,PDMR,xenograft,834989-109-R-G5YVH2,1.0,NaN,mutation,Illumina HiSeq 2000/2500,False,False,3350074490895,NaN
241,395191-088-T,PDMR,xenograft,395191-088-T-H5QU77W68,2.0,NaN,mutation,Illumina HiSeq 2000/2500,False,False,3375844294660,NaN
242,K41856-061-R2,PDMR,xenograft,K41856-061-R2-C61FU2,25.0,NaN,mutation,Illumina HiSeq 2000/2500,False,False,1812476198916,NaN
243,428932-153-R,PDMR,patient,428932-153-R,NaN,ERR4627966,mutation,Illumina HiSeq 2000/2500,False,False,816043786241,"[{""column"": ""raw_data_url"", ""resource"": ""ENA"",..."


In [57]:
for pro in missing_ids.data_source.unique():
    pro_path = join(home, pro)
    print(pro)
    missing_meta = missing_ids[missing_ids['data_source'] == pro]
    sample = read_metadata_with_fields(join(pro_path, pro+"_molecular_metadata-sample.tsv"))
    platform = read_metadata_with_fields(join(pro_path, pro+"_molecular_metadata-platform.tsv"))
    joined = sample.merge(platform[["platform_id", "molecular_characterisation_type", "instrument_model"]], on='platform_id', how='left')
    missing_meta['molecular_characterisation_type'], missing_meta['instrument_model'] = missing_meta['data_type'], missing_meta['platform_name']
    joined = joined.merge(missing_meta[['sample_id', 'molecular_characterisation_type', 'instrument_model', 'data_exists']], on =['sample_id', 'molecular_characterisation_type', 'instrument_model'], how='left', indicator=True)
    joined = joined[joined['_merge'] == 'left_only']    #joined = joined[~joined['sample_id'].isin(missing_meta['sample_id'])]
    joined = joined[sample.columns].reset_index(drop=True)
    joined.to_csv(join(pro_path, pro+"_molecular_metadata-sample.tsv"), sep='\t', index=False)

CRL
Curie-BC
IRCC-CRC
JAX
LIH
PDMR


/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_23946/4241553392.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_meta['molecular_characterisation_type'], missing_meta['instrument_model'] = missing_meta['data_type'], missing_meta['platform_name']
/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_23946/4241553392.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_meta['molecular_characterisation_type'], missing_meta['instrument_model'] = missing_meta['data_type'], miss

In [55]:
for pro in missing_ids.data_source.unique():
    pro_path = join(home, pro)
    print(pro)
    missing_meta = missing_ids[missing_ids['data_source'] == pro]
    sample = read_metadata_with_fields(join(pro_path, pro+"_molecular_metadata-sample.tsv"))
    platform = read_metadata_with_fields(join(pro_path, pro+"_molecular_metadata-platform.tsv"))
    joined = sample.merge(platform[["platform_id", "molecular_characterisation_type", "instrument_model"]], on='platform_id', how='left')
    missing_meta['molecular_characterisation_type'], missing_meta['instrument_model'] = missing_meta['data_type'], missing_meta['platform_name']
    joined = joined.merge(missing_meta[['sample_id', 'molecular_characterisation_type', 'instrument_model', 'data_exists']], on =['sample_id', 'molecular_characterisation_type', 'instrument_model'], how='left', indicator=True)
    joined = joined[joined['_merge'] == 'left_only']    #joined = joined[~joined['sample_id'].isin(missing_meta['sample_id'])]
    joined = joined[sample.columns].reset_index(drop=True)
    joined.to_csv(join(pro_path, pro+"_molecular_metadata-sample.tsv"), sep='\t', index=False)

CRL
['mutation_Illumina_WES' 'copy_number_alteration_Affymetrix_SNP6.0'
 'mutation_RNA_sequencing' 'expression_Affymetrix_HGU133plus2'
 'expression_ILLUMINA' 'cytogenetics_Immunohistochemistry']
(1910, 10)
(1910, 12)
(1895, 10)
Curie-BC
['cytogenetics_immunohistochemistry' 'mutation_NGS_ESOPE'
 'mutation_NGS_DRAGON']
(85, 10)
(85, 12)
(56, 10)
IRCC-CRC
['expression_Illumina_HT-12_v4_microarray' 'mutation_TargetedNGS_MUT'
 'mutation_whole_exome_sequencing'
 'copy_number_alteration_whole_exome_sequencing']
(1407, 10)
(1407, 12)
(1404, 10)
JAX
['mutation_CTP' 'mutation_Whole_Exome' 'mutation_Truseq_JAX'
 'mutation_Other:_ddPCR' 'mutation_Actionseq20' 'mutation_Other:_FISH'
 'mutation_TEX_DFCI' 'mutation_CTPTN' 'mutation_Other:_DFCI_RHP20'
 'copy_number_alteration_SNP' 'expression_RNA_Seq' 'expression_stRNA_Seq'
 'expression_hg10st' 'expression_hu133']
(1722, 10)
(1722, 12)
(1605, 10)
LIH
['copy_number_alteration_Agilent_aCGH_2x400k'
 'copy_number_alteration_Agilent_aCGH_4x180k' 'mutation_

/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_23946/2803976612.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_meta['molecular_characterisation_type'], missing_meta['instrument_model'] = missing_meta['data_type'], missing_meta['platform_name']
/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_23946/2803976612.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_meta['molecular_characterisation_type'], missing_meta['instrument_model'] = missing_meta['data_type'], mi

In [9]:
normal = pd.read_csv("/Users/tushar/Downloads/TCGA-LUAD.htseq_fpkm.tsv", sep='\t').iloc[:,0:15]

In [14]:
normal[normal['Ensembl_ID'].str.contains("133703")].reset_index(drop=True).to_csv("/Users/tushar/Downloads/Normal-FPKM-COLON.tsv",sep='\t',index=False)

In [ ]:
ENSG00000146648.14
ENSG00000171094.14


In [12]:
normal[normal['Ensembl_ID'].isin(["ENSG00000146648.14", "ENSG00000171094.14"])].reset_index(drop=True).to_csv("/Users/tushar/Downloads/Normal-FPKM-LUNG.tsv",sep='\t',index=False)

# Immune markers 
### Addition of immune markers -specifically model genomics in the data as not provided
MSI, TMB, MMR, Ploidy, Mutations per mb
- MSI: immune_msi
- MMR: immune_mmr 
- TMB: immune_tmb
- MPM: immune_mpm
- Ploidy: immune_ploidy

[['PDMR', ['immune_msi', 'immune_HLA_type']],
 ['JAX', ['immune_tmb', 'immune_msi']],
 ['CMP', ['immune_mpm', 'immune_ploidy', 'immune_msi']],
 ['CUIMC', ['immune_tmb']]]
 

[['PDMR', ['immune_msi', 'immune_HLA_type']],
 ['CRL', ['immune_HLA_type']],
 ['JAX', ['immune_tmb', 'immune_msi']],
 ['SJCRH', ['immune_hla_type']],
 ['CMP', ['immune_mpm', 'immune_ploidy', 'immune_msi']],
 ['DFCI-CPDM', ['immune_HLA_type']],
 ['CUIMC', ['immune_tmb']]]


In [93]:
model_genomics_markers = pd.read_json('https://dev.cancermodels.org/api/immunemarker_data_extended?marker_type=eq.Model%20Genomics')

In [95]:
marker_type = list(model_genomics_markers.marker_name.unique())
na_str = "Not provided"
marker_type.append('MMR')
marker_type_platform_dict = {'MSI': 'immune_msi', 'MMR': 'immune_mmr', 
                             'TMB': 'immune_tmb', 'Mutations per mb': 'immune_mpm', 
                             'Ploidy': 'immune_ploidy'}
platform_marker_type_dict = dict(zip(marker_type_platform_dict.values(), marker_type_platform_dict.keys()))
providers = list(model_genomics_markers.data_source.unique())
platform_ids = []

In [134]:
def get_available_marker_for_sample_id(sample_id, df, platform_list):
    temp = df[df['sample_id'] == sample_id]
    pids = temp['platform_id'].unique()
    add_markers = [x for x in platform_list if x not in pids]
    return [[sample_id, x] for x in add_markers]
    
    
def get_ms_row(sample_id, df, platform):
    df2 = df
    df = df.drop_duplicates(subset=['model_id', 'sample_id']).reset_index(drop=True)
    df = df[df['sample_id'] == sample_id].reset_index(drop=True)
    try:
        mid = df['model_id'][0]
    except:
        print(df)
        print(sample_id)
        print(df2[sample_id].unique())
    so = df['sample_origin'][0]
    pa = df['passage'][0]
    hsn = df['host_strain_name'][0]
    hsno = df['host_strain_nomenclature'][0]
    etcs = df['engrafted_tumor_collection_site'][0]
    rdu = df['raw_data_url'][0]
    #print(mid)
    
    # ['Field', 'model_id', 'sample_id', 'sample_origin', 'passage', 'host_strain_name', 'host_strain_nomenclature', 'engrafted_tumor_collection_site', 'raw_data_url', 'platform_id']
    temp = ['', mid, sample_id, so, pa, hsn, hsno, etcs, rdu, platform]
    return temp
    
def get_im_row(sample_id, na_str, marker_dict, platform):
    #['Field', 'sample_id', 'marker_type', 'marker_name', 'marker_value', 'essential_or_additional_details', 'platform_id']
    marker_name = marker_dict[platform]
    temp = ['', sample_id, 'Model Genomics', marker_name, na_str, '', platform]
    return temp
    
def append_to_df(row, df):
    row = pd.DataFrame([row], columns=df.columns)
    return pd.concat([df, row]).reset_index(drop=True)
            
for i in range(len(providers)):
    provider = providers[i]
    print(provider)
    provider_path = join(home, provider)
    mol_sample_sheet = read_metadata_with_fields(join(provider_path, provider+"_molecular_metadata-sample.tsv"))
    mol_platform = read_metadata_with_fields(join(provider_path, provider+"_molecular_metadata-platform.tsv"))
    mol_platform_web = read_metadata_with_fields(join(provider_path, provider+"_molecular_metadata-platform_web.tsv"))
    immunemarker = read_metadata_with_fields(join(provider_path, 'immunemarker', provider+"_immunemarker-Sheet1.tsv"))

    pids_in_sheet = list(immunemarker.platform_id.unique())
    add_markers = [x for x in marker_type_platform_dict.values() if x not in pids_in_sheet]
    print(add_markers)
    if len(add_markers)>0:
        sample_ids = immunemarker['sample_id'].unique()
        for j in tqdm(range(len(sample_ids)), f'Adding immune marker data for {provider}: '):
            sample_id = sample_ids[j]
            to_be_added = get_available_marker_for_sample_id(sample_id, immunemarker, marker_type_platform_dict.values())
            for row in to_be_added:
                immunemarker = append_to_df(get_im_row(row[0], na_str, platform_marker_type_dict, row[1]), immunemarker)            
                mol_sample_sheet = append_to_df(get_ms_row(row[0], mol_sample_sheet, row[1]), mol_sample_sheet)
        for mark in add_markers:
            mol_platform = append_to_df(['', mark, 'immunemarker', na_str, na_str, '', '', ''], mol_platform)
            mol_platform_web = append_to_df(['', mark, '', '', ''], mol_platform_web)
    
    mol_sample_sheet.to_csv(join(provider_path, provider+"_molecular_metadata-sample.tsv"), sep='\t', index=False)
    mol_platform.to_csv(join(provider_path, provider+"_molecular_metadata-platform.tsv"), sep='\t', index=False)
    mol_platform_web.to_csv(join(provider_path, provider+"_molecular_metadata-platform_web.tsv"), sep='\t', index=False)
    immunemarker.to_csv(join(provider_path, 'immunemarker', provider+"_immunemarker-Sheet1.tsv"), sep='\t', index=False)
    
        
    platform_id = mol_sample_sheet['platform_id'].iloc[4:].unique()
    platform_id =  [x for x in platform_id if x.__contains__('immune')]
    platform_ids.append([provider, platform_id])

PDMR
['immune_mmr', 'immune_tmb', 'immune_mpm', 'immune_ploidy']


Adding immune marker data for PDMR: 100%|██████████| 510/510 [00:22<00:00, 23.17it/s]


JAX
['immune_mmr', 'immune_mpm', 'immune_ploidy']


Adding immune marker data for JAX: 100%|██████████| 296/296 [00:06<00:00, 43.74it/s]


CMP
['immune_mmr', 'immune_tmb']


Adding immune marker data for CMP: 100%|██████████| 1174/1174 [00:32<00:00, 36.55it/s]


CUIMC
['immune_msi', 'immune_mmr', 'immune_mpm', 'immune_ploidy']


Adding immune marker data for CUIMC: 100%|██████████| 32/32 [00:00<00:00, 35.11it/s]


In [130]:
mol_platform_web.columns

Index(['Field', 'platform_id', 'molecular_methods_description',
       'analysis_description', 'data_acquisition_description', 'table'],
      dtype='object')

In [132]:
providers

['PDMR', 'JAX', 'CMP', 'CUIMC']

In [71]:
from os import listdir
from os.path import isfile, join
import pandas as pd

CMP_path = '/hps/nobackup/tudor/pdcm/pdxfinder-data/data/UPDOG/CMP/'
CMP_mut_path = '/hps/nobackup/tudor/pdcm/pdxfinder-data/data/UPDOG/CMP/mut'

def get_files(path):
    return [join(path, f) for f in listdir(path) if isfile(join(path, f))]

def read_metadata_with_fields(path):
    metadata = pd.read_csv(path, sep='\t', na_values="", low_memory=False)
    return metadata

def read_metadata_without_fields(path):
    metadata = pd.read_csv(path, sep='\t', na_values="", low_memory=False)
    if 'Field' in metadata.columns:
        metadata = metadata.loc[metadata.Field.str.startswith('#') != True,].reset_index(drop=True)
        metadata = metadata.drop('Field', axis=1)
    return metadata

#mms_header = read_metadata_with_fields(join(CMP_path, 'CMP_molecular_metadata-sample.tsv')).iloc[0:4]
#ps = read_metadata_without_fields(join(CMP_path, 'CMP_molecular_metadata-sample.tsv'))[['sample_id', 'model_id']]
#ps = ps.drop_duplicates().reset_index(drop=True)
#ps = dict(zip(ps['sample_id'], ps['model_id']))
mms = read_metadata_without_fields(join(CMP_path, 'CMP_molecular_metadata-sample.tsv'))[['sample_id', 'model_id']]
mms = mms.drop_duplicates().reset_index(drop=True)
mms_mapper = dict(zip(mms['sample_id'], mms['model_id']))
#replaced_mms = mms[['model_id', 'sample_id']].replace(ps)
#mms['model_id'] = replaced_mms['sample_id']
mut_file = read_metadata_with_fields(join(CMP_mut_path, 'CMP_mut.tsv'))
samples = mms['sample_id'].unique()
for i in tqdm(range(len(samples))):
    s = samples[i]
    temp = mut_file[mut_file['sample_id'] == s].reset_index(drop=True)
    if len(temp) > 0:
        file_name = join(CMP_mut_path, f"CMP_mut_{s}.tsv")
        temp.to_csv(file_name, sep='\t', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/hps/nobackup/tudor/pdcm/pdxfinder-data/data/UPDOG/CMP/CMP_molecular_metadata-sample.tsv'

In [20]:
from utils import read_metadata_with_fields, read_metadata_without_fields, get_files
from os.path import join, exists
from os import remove
from pandas import read_csv as read, concat, to_numeric
GCCRI_path = '/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/GCCRI'

In [30]:
pivot_id_path = '~/CancerModels/submission/PIVOT_IDs.csv'
pivot_ids = read(pivot_id_path, encoding='utf-8')
mapper = {'LURIE': 'LurieChildrens', 'MDA': 'MDAnderson-CCH', 'UTHSCSA': 'GCCRI'}
pivot_ids['provider'] = pivot_ids['PIVOT Center (Model Originator)'].replace(mapper)
pivot_providers = list(pivot_ids['provider'].unique())
GCCRI = pivot_ids[pivot_ids['provider'] == 'GCCRI']

In [8]:
pdx = read_metadata_without_fields(join(GCCRI_path, 'GCCRI_metadata-pdx_model.tsv'))
new_model_id = [mid.replace('-', '').replace('NCH', 'NCH-').replace('1NCH', '1-NCH').replace('2NCH', '2-NCH') for mid in pdx['model_id']]
new_model_id = [mid.replace('S13', '-S13_').replace('--', '-') for mid in new_model_id]
mapper = dict(zip(list(pdx['model_id']), new_model_id))
ps_mapper = dict(zip(list(pdx['model_id'] + '_P'), [n + '_P' for n in new_model_id]))
#pdx.replace(mapper).to_csv(join(GCCRI_path, 'GCCRI_metadata-pdx_model.tsv'), sep='\t', index=False)

In [9]:
def fix_ids(path, mapper):
    df = read_metadata_without_fields(path).replace(mapper)
    header = read_metadata_with_fields(path).iloc[0:4]
    return concat([header, df]).reset_index(drop=True) 

In [10]:
fix_ids(join(GCCRI_path, 'GCCRI_metadata-pdx_model.tsv'), mapper).to_csv(join(GCCRI_path, 'GCCRI_metadata-pdx_model.tsv'), sep='\t', index=False)

In [67]:
fix_ids(join(GCCRI_path, 'GCCRI_metadata-sharing.tsv'), mapper).to_csv(join(GCCRI_path, 'GCCRI_metadata-sharing.tsv'), sep='\t', index=False)

In [71]:
fix_ids(join(GCCRI_path, 'GCCRI_metadata-model_validation.tsv'), mapper).to_csv(join(GCCRI_path, 'GCCRI_metadata-model_validation.tsv'), sep='\t', index=False)

,Field,model_id,validation_technique,description,passages_tested,validation_host_strain_nomenclature
0,#Description,Unique identifier for all the PDXs derived fro...,Any technique used to validate PDX against the...,Short description of what was compared and wha...,Provide a list of all passages where validatio...,"Validation host mouse strain, following mouse ..."
1,#Example,CRC0228PR,fingerprinting,high concordance between xenograft and patien...,"1,2",NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ
2,#Format Requirements,free text,free text,free text,list of numbers separted by commas,full host strain name or Not provided
3,#Essential?,essential,essential,essential,essential,essential
4,NaN,ASPSKY,Fingerprinting,Not provided,Not provided,C.B-Igh-1b/IcrTac-Prkdc<sup>scid</sup>
5,NaN,BT27,Fingerprinting,Not provided,Not provided,C.B-Igh-1b/IcrTac-Prkdc<sup>scid</sup>
6,NaN,BT29,Fingerprinting,Not provided,Not provided,C.B-Igh-1b/IcrTac-Prkdc<sup>scid</sup>
7,NaN,ES1,Fingerprinting,Not provided,Not provided,C.B-Igh-1b/IcrTac-Prkdc<sup>scid</sup>
8,NaN,ES4,Fingerprinting,Not provided,Not provided,C.B-Igh-1b/IcrTac-Prkdc<sup>scid</sup>
9,NaN,ES6,Fingerprinting,Not provided,Not provided,C.B-Igh-1b/IcrTac-Prkdc<sup>scid</sup>


In [82]:
ps_mapper = dict(zip(list(pdx['model_id'] + '_P'), [n + '_P' for n in new_model_id]))
out = fix_ids(join(GCCRI_path, 'GCCRI_metadata-patient_sample.tsv'), mapper)
out.replace(ps_mapper).to_csv(join(GCCRI_path, 'GCCRI_metadata-patient_sample.tsv'), sep='\t', index=False)

In [83]:
out = fix_ids(join(GCCRI_path, 'GCCRI_molecular_metadata-sample.tsv'), mapper)
out.replace(ps_mapper).to_csv(join(GCCRI_path, 'GCCRI_molecular_metadata-sample.tsv'), sep='\t', index=False)

In [87]:
mut = read(join(GCCRI_path, 'mut/GCCRI_mut.tsv'), sep='\t').replace(ps_mapper)
mut.to_csv(join(GCCRI_path, 'mut/GCCRI_mut.tsv'), sep='\t', index=False)

In [85]:
mut

,sample_id,symbol,biotype,coding_sequence_change,variant_class,codon_change,amino_acid_change,consequence,functional_prediction,read_depth,...,seq_start_position,ref_allele,alt_allele,ucsc_gene_id,ncbi_gene_id,ncbi_transcript_id,ensembl_gene_id,ensembl_transcript_id,variation_id,platform_id
0,ASPS-KY_P,ACAD8,protein_coding,557A>G,SNV,aAt/aGt,N186S,missense_variant,NaN,NaN,...,134259074,A,G,NaN,NaN,NaN,ENSG00000151498,ENST00000281182,rs200170162,mutation_WES
1,ASPS-KY_P,ACHE,protein_coding,827G>A,SNV,cGt/cAt,R276H,missense_variant,NaN,NaN,...,100893406,C,T,NaN,NaN,NaN,ENSG00000087085,ENST00000241069,rs533264163,mutation_WES
2,ASPS-KY_P,ACOX3,protein_coding,1306C>T,SNV,Cgg/Tgg,R436W,missense_variant,NaN,NaN,...,8389729,G,A,NaN,NaN,NaN,ENSG00000087008,ENST00000356406,rs146052311&COSV62712425,mutation_WES
3,ASPS-KY_P,ACTN1,protein_coding,1822C>T,SNV,Cgg/Tgg,R608W,missense_variant,NaN,NaN,...,68882589,G,A,NaN,NaN,NaN,ENSG00000072110,ENST00000394419,rs1468329163&COSV99518838,mutation_WES
4,ASPS-KY_P,ADARB2,protein_coding,283G>T,SNV,Ggc/Tgc,G95C,missense_variant,NaN,NaN,...,1363822,C,A,NaN,NaN,NaN,ENSG00000185736,ENST00000381312,rs748408152&COSV105323616&COSV67231399,mutation_WES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7002,TC-71_P,LRRC55,protein_coding,407C>T,SNV,gCc/gTc,A136V,missense_variant,NaN,NaN,...,57182429,C,T,NaN,NaN,NaN,ENSG00000183908,ENST00000497933,COSV73006284,mutation_WES
7003,TC-71_P,ZNF728,protein_coding,143C>T,SNV,cCt/cTt,P48L,missense_variant,NaN,NaN,...,22987391,G,A,NaN,NaN,NaN,ENSG00000269067,ENST00000594710,rs1157096293,mutation_WES
7004,TC-71_P,IL27RA,protein_coding,NaN,SNV,NaN,NaN,downstream_gene_variant,NaN,NaN,...,14055064,G,A,NaN,NaN,NaN,ENSG00000104998,ENST00000263379,rs201934041&COSV54602528,mutation_WES
7005,TC-71_P,FLG2,protein_coding,5830G>A,SNV,Ggg/Agg,G1944R,missense_variant,NaN,NaN,...,152351956,C,T,NaN,NaN,NaN,ENSG00000143520,ENST00000388718,rs200946758,mutation_WES


In [89]:
treatment = read(join(GCCRI_path, 'treatment/GCCRI_patienttreatment-Sheet1.tsv'), sep='\t').replace(mapper)
treatment.to_csv(join(GCCRI_path, 'treatment/GCCRI_patienttreatment-Sheet1.tsv'), sep='\t', index=False)

In [102]:
exists(join(GCCRI_path, 'mut'))

True

In [27]:
def convert_to_int_or_blank(value):
    try:
        return int(float(value))
    except ValueError:
        return ''
    
exp = get_files(join(GCCRI_path, 'expression'))
exp = [f for f in exp if f.endswith('.tsv')]
for f in exp:
    temp = read(join(GCCRI_path, 'expression', f), sep='\t')
    new_f = mapper[f.split('_')[2].replace('.tsv', '')]
    new_f = 'GCCRI_expression_' + new_f + '.tsv'
    new_f = join(GCCRI_path, 'expression', new_f)
    temp['ensembl_gene_id'] = [convert_to_int_or_blank(p)  for p in temp['ensembl_gene_id']]
    temp.replace(ps_mapper).to_csv(new_f, sep='\t', index=False)
    remove(join(GCCRI_path, 'expression', f))

In [33]:
cna = get_files(join(GCCRI_path, 'cna'))
cna = [f for f in cna if f.endswith('.tsv')]
for f in cna:
    temp = read(join(GCCRI_path, 'cna', f), sep='\t')
    new_f = mapper[f.split('_')[2].replace('.tsv', '')]
    new_f = 'GCCRI_cna_' + new_f + '.tsv'
    new_f = join(GCCRI_path, 'cna', new_f)
    #temp['ensembl_gene_id'] = [convert_to_int_or_blank(p)  for p in temp['ensembl_gene_id']]
    temp.replace(ps_mapper).to_csv(new_f, sep='\t', index=False)
    remove(join(GCCRI_path, 'cna', f))

In [31]:
f

'GCCRI_cna_Rh-30R.tsv'

In [4]:
import pandas as pd
df = pd.read_json('https://www.cancermodels.org/api/model_metadata?select=data_source,provider_name,contact_name_list,contact_email_list,contact_form_url').drop_duplicates(subset=['data_source', 'contact_name_list'])
df

,data_source,provider_name,contact_name_list,contact_email_list,contact_form_url
0,JAX,The Jackson laboratory,micetech,micetech@jax.org,http://tumor.informatics.jax.org/mtbwi/pdxRequ...
1,PDMR,Patient-Derived Model Repository,Pete Hilfiker,hilfikerp@mail.nih.gov,https://pdmr.cancer.gov/request/default.htm
2,DFCI-CPDM,"Center for Patient Derived Models, Dana-Farber...",Abigail Ossip,dfcipatientderivedmodels@partners.org,https://www.dana-farber.org/research/departmen...
4,MDAnderson,MD Anderson Cancer Center,Not Provided,bfang@mdanderson.org,None
6,CRL,Charles River Laboratories,"Julia Schueler, Hagen Klett",tumor-model-compendium@crl.com,https://www.criver.com/resources/tumor-model-d...
9,IRCC-CRC,Candiolo Cancer Institute - Colorectal,andrea bertotti,andrea.bertotti@ircc.it,None
11,GCCRI,UT Health San Antonio Greehey Children's Cance...,Peter J Houghton,HoughtonP@uthscsa.edu,https://preclinicalpivot.org/contact-us/
12,VHIO-CRC,Val D'Hebron Cancer Institute- colorectal,"Lorena Ramírez, Irene Chicote, Héctor G. Palmer","lramirez@vhio.net, ichicote@vhio.net, hgpalmer...",None
14,CRL,Charles River Laboratories,Julia Schueler,tumor-model-compendium@crl.com,https://www.criver.com/resources/tumor-model-d...
26,Wistar-MDAnderson-Penn,Wistar-MDAnderson-Penn,"Meenhard Herlyn, Michael A. Davies","herlynm@Wistar.org,MDavies@mdanderson.org",None


In [8]:
df.sort_values(by='data_source').reset_index(drop=True)

,data_source,provider_name,contact_name_list,contact_email_list,contact_form_url
0,BROD,Broad Institute,NCI CC Genomics,NCICCGenomics@mail.nih.gov,None
1,CCIA,Children's Cancer Institute,Richard Lock,RLock@ccia.org.au,https://preclinicalpivot.org/contact-us/
2,CHOP,Children's Hospital of Philadelphia,"Reynolds, Maris","patrick.reynolds@ttuhsc.edu,maris@email.chop.edu",https://preclinicalpivot.org/contact-us/
3,CMP,Cell Model Passports,Sanger DepMap,depmap@sanger.ac.uk,None
4,CRL,Charles River Laboratories,Julia Schueler,tumor-model-compendium@crl.com,https://www.criver.com/resources/tumor-model-d...
5,CRL,Charles River Laboratories,"Julia Schueler, Hagen Klett",tumor-model-compendium@crl.com,https://www.criver.com/resources/tumor-model-d...
6,CSHL,Cold Spring Harbor Laboratory,NCI CC Genomics,NCICCGenomics@mail.nih.gov,None
7,CUIMC,Columbia University Irving Medical Center,Michael M. Shen,mshen@columbia.edu,None
8,Curie-BC,Institut Curie - LIP - BC,Elisabetta Marangoni,elisabetta.marangoni@curie.fr,None
9,Curie-LC,Institut Curie - LIP - LC,Didier Decaudin; Elodie Montaudon,"didier.decaudin@curie.fr, elodie.montaudon@cur...",None


In [32]:
providers = sorted(get_dirs(home))
cbioportal_case_lists = "/Users/tushar/CancerModels/utils/cbioportal/pdcm-cbioportal/study"
for provider in providers:
    case_list = join(cbioportal_case_lists, provider, 'case_lists/cases_all.txt')
    case_file = pd.DataFrame(pd.read_csv(case_list, sep=': ', engine='python', skiprows=range(0, 4)).iloc[0,1].split('\t'), columns=['sample_id'])
    if not exists(join(home, provider, 'case_lists')):
        makedirs(join(home, provider, 'case_lists'))
    out_case_list = join(home, provider, 'case_lists/cases_all.tsv')
    case_file.to_csv(out_case_list, sep='\t', index=False)

In [27]:
case_file

,0
0,HCM-BROD-0781-C71-85A-01R-A88J-41
1,HCM-BROD-0676-C71-85R-01R-A85D-41
2,HCM-BROD-0003-C71-86A-01R-A78N-41
3,HCM-BROD-0783-C71_sample
4,HCM-BROD-0449-C34-85M-01R-A82J-41
...,...
466,HCM-BROD-0199-C71_sample
467,HCM-BROD-0122-C25-85A-01D-A79L-36
468,HCM-BROD-0477-C16_sample
469,HCM-BROD-0013-C71_sample


## Data model changes

In [5]:
providers = sorted(get_dirs(home))
template_path = "/Users/tushar/CancerModels/pdxfinder-data/template/active_templates/metadata"
templates = [f for f in sorted(get_files(template_path)) if f.endswith('tsv') and not f.__contains__('image') and not f.__contains__('sharing')]
for provider in providers:
    for template in templates:
        template_df = read_metadata_with_fields(join(template_path, template))
        sheet_name = join(home, provider, f"{provider}_{template.replace('_template', '')}")
        if exists(sheet_name):
            sheet_df = read_metadata_without_fields(sheet_name)
            sheet_df['Field'] = ""
            if sheet_df.shape[0] == 0 and template.__contains__('validation'):
                sheet_df = template_df
            else:
                for col in template_df.columns:
                    if col not in sheet_df.columns:
                        if col == 'parent_id' or col ==	'origin_patient_sample_id':
                            sheet_df.loc[:, col] = ""
                        else:
                            sheet_df.loc[:, col] = "Not provided"
                #if sheet_name.__contains__("cell"):
                #    sheet_df['model_name'] = sheet_df['name']
                sheet_df = pd.concat([template_df, sheet_df[template_df.columns]])
            sheet_df.to_csv(sheet_name, sep='\t', index=False)

In [7]:
template

'metadata_template-patient_sample.tsv'

In [17]:
template_path

'/Users/tushar/CancerModels/pdxfinder-data/template/active_templates/metadata'

In [18]:
providers

['BROD',
 'CCIA',
 'CHOP',
 'CMP',
 'CRL',
 'CSHL',
 'CUIMC',
 'Curie-BC',
 'Curie-LC',
 'Curie-OC',
 'DFCI-CPDM',
 'GCCRI',
 'HCI-BCM',
 'HKU',
 'IRCC-CRC',
 'IRCC-GC',
 'JAX',
 'LIH',
 'LurieChildrens',
 'MDAnderson',
 'MDAnderson-CCH',
 'NKI',
 'PDMR',
 'PMLB',
 'PMLB-Organoid',
 'SANG',
 'SJCRH',
 'TRACE',
 'UCD',
 'UMCG',
 'UOC-BC',
 'UOM-BC',
 'VHIO-BC',
 'VHIO-CRC',
 'VHIO-PC',
 'WCMC',
 'WUSTL',
 'Wistar-MDAnderson-Penn']

In [17]:
from requests import post
import json

In [4]:
normal_data = pd.read_csv("/Users/tushar/Downloads/tcga_normal_fpkm.tsv", sep='\t')
normal_samples = pd.read_csv("/Users/tushar/Downloads/TCGA_phenotype_denseDataOnlyDownload.tsv", sep='\t')
url = "https://biotools.fr/human/ensembl_symbol_converter/"
response = post(url, data={'api':1, 'ids':str(list(normal_data['sample'].str.split('.').str[0])).replace('\'', '\"')})
ensembl2genesymbol = response.json()
normal_data['gene_symbol'] = normal_data['sample'].str.split('.').str[0].replace(ensembl2genesymbol)

In [5]:
sorted(normal_samples['_primary_disease'].unique())

['bladder urothelial carcinoma',
 'breast invasive carcinoma',
 'cervical & endocervical cancer',
 'cholangiocarcinoma',
 'colon adenocarcinoma',
 'esophageal carcinoma',
 'glioblastoma multiforme',
 'head & neck squamous cell carcinoma',
 'kidney chromophobe',
 'kidney clear cell carcinoma',
 'kidney papillary cell carcinoma',
 'liver hepatocellular carcinoma',
 'lung adenocarcinoma',
 'lung squamous cell carcinoma',
 'ovarian serous cystadenocarcinoma',
 'pancreatic adenocarcinoma',
 'pheochromocytoma & paraganglioma',
 'prostate adenocarcinoma',
 'rectum adenocarcinoma',
 'sarcoma',
 'skin cutaneous melanoma',
 'stomach adenocarcinoma',
 'thymoma',
 'thyroid carcinoma',
 'uterine corpus endometrioid carcinoma']

In [64]:
normal_data['gene_symbol'] = normal_data['sample'].str.split('.').str[0].replace(ensembl2genesymbol) 

In [70]:
breast_samples = ['gene_symbol'] + list(normal_samples[normal_samples["_primary_disease"] == "breast invasive carcinoma"]['sample'])
breast_samples = [c for c in breast_samples if c in normal_data.columns]
normal_data[breast_samples].dropna().to_csv("/Users/tushar/Downloads/tcga_brca_normal_fpkm.tsv", sep='\t',index=False)

In [69]:
lung_samples = ['gene_symbol'] + list(normal_samples[normal_samples["_primary_disease"] == "lung adenocarcinoma"]['sample'])
lung_samples = [c for c in lung_samples if c in normal_data.columns]
normal_data[lung_samples].dropna().to_csv("/Users/tushar/Downloads/tcga_lung_normal_fpkm.tsv", sep='\t',index=False)

In [68]:
colon = ['gene_symbol'] + list(normal_samples[normal_samples["_primary_disease"] == "colon adenocarcinoma"]['sample'])
colon = [c for c in colon if c in normal_data.columns]
normal_data[colon].dropna().to_csv("/Users/tushar/Downloads/tcga_colon_normal_fpkm.tsv", sep='\t',index=False)

In [93]:
mms = read_metadata_without_fields(join(home, "CMP/CMP_molecular_metadata-sample.tsv"))
ps = read_metadata_without_fields(join(home, "CMP/CMP_metadata-patient_sample.tsv"))

In [94]:
#model_sample_mapper = mms[~mms['model_id'].str.contains("immune")][['model_id', 'sample_id']].drop_duplicates()
model_sample_mapper = dict(zip(ps['sample_id'], ps['model_id']))

missing = list()
for i in range(mms.shape[0]):
    if str(mms['model_id'][i]).__contains__('immune'):
        if mms['sample_id'][i] not in model_sample_mapper.keys():
            #print(mms['sample_id'][i])
            missing.append(mms['sample_id'][i])
        else:
            mms['model_id'][i] = model_sample_mapper[mms['sample_id'][i]]


In [95]:
len(set(missing))    

0

In [96]:
set(missing)

set()

In [97]:
mms

,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,SIDM00993,SIDS00452,cell,NaN,NaN,NaN,NaN,NaN,immune_msi
1,SIDM00993,SIDS00452,cell,NaN,NaN,NaN,NaN,NaN,immune_mpm
2,SIDM00993,SIDS00452,cell,NaN,NaN,NaN,NaN,NaN,immune_ploidy_wes
3,SIDM00993,SIDS00452,cell,NaN,NaN,NaN,NaN,NaN,immune_HLA_type
4,SIDM00993,SIDS00452,cell,NaN,NaN,NaN,NaN,NaN,immune_tmb
...,...,...,...,...,...,...,...,...,...
13076,SIDM01779,SIDS01654,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid
13077,SIDM01780,SIDS01653,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid
13078,SIDM01781,SIDS01664,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid
13079,SIDM01782,SIDS01678,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid


In [98]:
out_mms = read_metadata_with_fields(join(home, "CMP/CMP_molecular_metadata-sample.tsv"))

In [100]:
out_mms.iloc[0:4]

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,desirable,desirable,essential


In [101]:
mms['Field'] = ""

In [106]:
pd.concat([out_mms.iloc[0:4], mms]).to_csv(join(home, "CMP/CMP_molecular_metadata-sample.tsv"), sep='\t', index=False)

In [105]:
out_mms

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,desirable,desirable,essential
4,NaN,immune_msi,SIDS00452,cell,NaN,NaN,NaN,NaN,NaN,immune_msi
...,...,...,...,...,...,...,...,...,...,...
13080,NaN,SIDM01779,SIDS01654,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid
13081,NaN,SIDM01780,SIDS01653,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid
13082,NaN,SIDM01781,SIDS01664,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid
13083,NaN,SIDM01782,SIDS01678,cell,NaN,NaN,NaN,NaN,NaN,cna_WGS_organoid


## MI Organoids

In [560]:
from requests import get
from re import search, sub
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import numpy as np

In [5]:
in_vitro_models = read_metadata_without_fields(join(home, 'BROD', 'BROD_metadata-cell_model.tsv'))

0       https://www.atcc.org/products/PDM-18
1       https://www.atcc.org/products/PDM-17
2       https://www.atcc.org/products/PDM-19
3       https://www.atcc.org/products/PDM-23
4       https://www.atcc.org/products/PDM-22
                       ...                  
226    https://www.atcc.org/products/PDM-171
227    https://www.atcc.org/products/PDM-166
228    https://www.atcc.org/products/PDM-173
229    https://www.atcc.org/products/PDM-495
230    https://www.atcc.org/products/PDM-165
Name: vendor_link, Length: 231, dtype: object

In [18]:
url = in_vitro_models['vendor_link'][0].lower()

In [263]:
def fetch_from_atcc(in_vitro_models):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode if desired
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    first = True
    text = pd.DataFrame()
    #attributes = list()
    for url in in_vitro_models['vendor_link'].str.lower():
        if url == 'not provided':
            continue
        driver.get(url)
        if first:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Allow all cookies']"))).click()
            first = False
        #print('Expand')
        sleep(2)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Expand all')]"))).click()
        headers = list()
        h3 = driver.find_elements(By.XPATH, "//h3")
        append_it = False
        for h in h3:
            if len(headers) == 0:
                headers.append('Product Information')
            if h.text == "General" or h.text == 'Characteristics' or append_it:
                headers.append(h.text)
                append_it = True
            if h.text == "Legal disclaimers":
                append_it = False
        
        dls = driver.find_elements(By.XPATH, "//dl")
        i = 0
        df = pd.DataFrame()
        df['atcc'] = [url.replace('https://www.atcc.org/products/', '').upper()]
        df['url'] = url
        for dl in dls:
            soup = BeautifulSoup(dl.get_attribute("outerHTML"), 'html.parser')
            dt_elements = soup.find_all('dt', class_='product-information__title')
            dd_elements = soup.find_all('dd', class_='product-information__data')
            dt_values = [headers[i].replace(' ', '_').lower()+'.'+dt.get_text(strip=True).replace(' ', '_').lower() for dt in dt_elements]
            i = i+1
            dd_values = [dd.get_text(strip=True) for dd in dd_elements]
            for j in range(len(dt_values)):
                df[dt_values[j]] = dd_values[j]  
        text = pd.concat([text, df]).reset_index(drop=True)
    driver.quit()
    return text

In [90]:
text

,atcc,url,product_information.product_category,product_information.product_type,product_information.organism,product_information.morphology,product_information.tissue,product_information.disease,product_information.applications,product_information.product_format,...,history.depositors,history.year_of_origin,history.special_collection,legal_disclaimers.intended_use,legal_disclaimers.warranty,legal_disclaimers.disclaimers,characteristics.gender,characteristics.metastatic,handling_information.cryopreservation,characteristics.passage_history
0,PDM-18,https://www.atcc.org/products/pdm-18,Human cells,Cell model,"Homo sapiens, human",neuronal,Brain,Glioblastoma; Primary,3D cell cultureCancer researchNeuroscience,Frozen,...,Broad Institute,2018,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN,NaN,NaN
1,PDM-17,https://www.atcc.org/products/pdm-17,Human cells,Cell model,"Homo sapiens, human",neuronal; conditionally reprogrammed cell (CRC),Brain,Glioblastoma; Primary,3D cell cultureCancer researchNeuroscience,Frozen,...,Broad Institute,2018,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN,NaN,NaN
2,PDM-19,https://www.atcc.org/products/pdm-19,Human cells,Cell model,"Homo sapiens, human",neuronal,Brain,Glioblastoma; Recurrent,3D cell cultureCancer researchNeuroscience,Frozen,...,Broad Institute,2018,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,Female,NaN,NaN,NaN
3,PDM-23,https://www.atcc.org/products/pdm-23,Human cells,Cell model,"Homo sapiens, human",neuronal,Brain,Glioblastoma; Recurrent,3D cell cultureCancer researchNeuroscience,Frozen,...,Broad Institute,2018,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN,NaN,NaN
4,PDM-22,https://www.atcc.org/products/pdm-22,Human cells,Cell model,"Homo sapiens, human",NaN,Brain,Glioblastoma; Recurrent,3D cell cultureCancer researchNeuroscience,Frozen,...,Broad Institute,2018,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,Male,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,PDM-171,https://www.atcc.org/products/pdm-171,Human cells,Cell modelOrganoid,"Homo sapiens, human",Organoid,Pancreas,Adenocarcinoma; Metastatic,3D cell cultureCancer research,Frozen,...,Broad Institute,2021,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,Metastatic,For a brief overview of the cryopreservation p...,NaN
140,PDM-166,https://www.atcc.org/products/pdm-166,Human cells,Cell modelOrganoid,"Homo sapiens, human",Organoid,Pancreas,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Broad Institute,2021,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN,For a brief overview of the cryopreservation p...,NaN
141,PDM-173,https://www.atcc.org/products/pdm-173,Human cells,Cell modelOrganoid,"Homo sapiens, human",Organoid,Pancreas,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Broad Institute,2021,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is i

In [215]:
conditions = [text['handling_information.complete_medium'].str.contains('NeuroCult'), 
              text['handling_information.complete_medium'].str.contains('Propagenix'), 
              text['handling_information.complete_medium'].str.contains('Renaissance'), 
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #3'), text['handling_information.complete_medium'].str.contains('Lonza SmGM'), 
              text['handling_information.complete_medium'].str.contains('ATCC 30-2001'), 
              text['handling_information.complete_medium'].str.contains('AR5 Media with 5% FBS'),
              text['handling_information.complete_medium'].str.contains('HCM-BROD-0051-C64'), 
              text['handling_information.complete_medium'].str.contains('Dulbecco'), 
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #5')]


replacements = ['Stem Cell Technologies - 5751', 'Propagenix 256-100', 'Cellaria Bio CM-0001', 'ATCC ACS-7101', 'Lonza CC-3182', 'ATCC 30-2001', 'Not provided', 'Propagenix 256-10', 'ATCC 30-2002', 'ATCC ACS-7104']


growth_media = ['NeuroCult™ NS-A Proliferation Kit (Human)', 'Propagenix Conditioned Medium', 'Renaissance Essential Tumor Medium', 'Organoid Growth Kit 1B', 'SmGM - 2 Smooth Muscle Cell Growth Medium - 2 BulletKit', 'RPMI-1640 Medium', 'AR5 Media with 5% FBS', 'Propagenix Conditioned Medium', 'DMEM', 'Organoid Growth Kit 1E']

supplements = ['NS-A Proliferation Supplement (StemCell Technologies #05754) + 20 ng/mL EGF (StemCell Technologies #78003.1) + 20 ng/mL bFGF (Peprotech #AF-100-15) + 2 µg/mL Heparin (StemCell Technologies #07980)',
                 '9.0 ng/mL cholera toxin (Sigma Aldrich C8052)', 
                 '5% Fetal Bovine Serum (FBS; ATCC 30-2020) and 25 ng/mL Cholera Toxin (Sigma Aldrich cat# C8052)', 
                 'Not provided', 
                 'Not provided', 
                 'Fetal Bovine Serum (FBS; ATCC 30-2020)', 
                 'RPMI with L-Glutamine: 5% FBS, 1% HEPES (1 M), 2 mg/mL BSA, 500 µMm Sodium Pyruvate, 20 µg/mL Insulin, 10 µM Ethanolamine, 0.1 nM 3,3’,5-Triiodo-LThyronine (T3), 30 nM Sodium Selenite, 10 µg/mL Transferrin, 10 ng/mL EGF, 10 µM O-Phosphorylethanolamine (PHOS), 50 nM Hydrocortisone',
                 '9.0 ng/mL cholera toxin',
                 '10% Fetal Bovine Serum (FBS;ATCC 30-2020)',
                 'Not provided']

text['media_id'] = np.select(conditions, replacements, default='Not provided')
text['growth_media'] = np.select(conditions, growth_media, default='Not provided')
text['supplements'] = np.select(conditions, supplements, default='Not provided')
text['growth_properties'] = text['characteristics.growth_properties']
text['contaminated'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'Yes', 'No')

text['contamination_details'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'No contamination detected', 'Not provided')

text['model_purity'] = np.where(text['characteristics.metastatic'].fillna('Not provided') == 'Not provided', 'Not provided', 'Tumor cells')

text['STR_analysis'] = np.where(text['quality_control_specifications.str_profiling'].fillna('Not provided') == 'Not provided', 'No', 'Yes')

text['morphological_features'] = text['product_information.morphology'].fillna('Not provided')

text['virology_status'] = np.where(text['quality_control_specifications.virus_testing'].str.lower().str.replace('detected', 'detected, ').str.replace('not detected', '').str.contains('detected'), 'Detected', 'Not detected')

In [216]:
cell_sheet = read_metadata_without_fields(join(home, 'BROD', 'BROD_metadata-cell_model.tsv'))
cell_template = read_metadata_with_fields(join(home, 'BROD', 'BROD_metadata-cell_model.tsv')).iloc[0:4]
merged = cell_sheet.merge(text, left_on='catalog_number', right_on='atcc', how='left', suffixes=('_og', '_supplier'))
merged['growth_properties'] = merged['growth_properties_supplier']
merged['media_id'] = merged['media_id_supplier']
merged['growth_media'] = merged['growth_media_supplier']
merged['contaminated'] = merged['contaminated_supplier']
merged['contamination_details'] = merged['contamination_details_supplier']
merged['supplements'] = merged['supplements_supplier']
pd.concat([cell_template, merged[cell_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'BROD', 'BROD_metadata-cell_model.tsv'), sep='\t', index=False)

Index(['model_id', 'model_name', 'model_name_aliases', 'type', 'parent_id',
       'origin_patient_sample_id', 'growth_properties_og', 'media_id_og',
       'growth_media_og', 'plate_coating', 'other_plate_coating',
       'passage_number', 'contaminated_og', 'contamination_details_og',
       'supplements_og', 'drug', 'drug_concentration', 'publications',
       'supplier', 'supplier_type', 'catalog_number', 'vendor_link', 'rrid',
       'external_ids', 'comments', 'atcc', 'url',
       'product_information.product_category',
       'product_information.product_type', 'product_information.organism',
       'product_information.morphology', 'product_information.tissue',
       'product_information.disease', 'product_information.applications',
       'product_information.product_format',
       'product_information.storage_conditions',
       'general.specific_applications', 'characteristics.cells_per_vial',
       'characteristics.volume', 'characteristics.growth_properties',
       'c

In [221]:
cell_sheet.columns

Index(['model_id', 'model_name', 'model_name_aliases', 'type', 'parent_id',
       'origin_patient_sample_id', 'growth_properties', 'media_id',
       'growth_media', 'plate_coating', 'other_plate_coating',
       'passage_number', 'contaminated', 'contamination_details',
       'supplements', 'drug', 'drug_concentration', 'publications', 'supplier',
       'supplier_type', 'catalog_number', 'vendor_link', 'rrid',
       'external_ids', 'comments'],
      dtype='object')

,model_id,model_name,model_name_aliases,type,parent_id,origin_patient_sample_id,growth_properties,media_id,growth_media,plate_coating,...,drug,drug_concentration,publications,supplier,supplier_type,catalog_number,vendor_link,rrid,external_ids,comments
0,HCM-BROD-0011-C71,HCM-BROD-0011-C71,Not provided,"3-D: Other (e.g. neurosphere, air-liquid inter...",NaN,NaN,Mixed: suspension and aggregate,Stem Cell Technologies - 5751,NeuroCult™ NS-A Proliferation Kit (Human),Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-18,https://www.atcc.org/products/PDM-18,Not provided,NaN,Not provided
1,HCM-BROD-0003-C71,HCM-BROD-0003-C71,Not provided,Cell Line,NaN,NaN,Adherent,Propagenix 256-100,Propagenix Conditioned Medium,Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-17,https://www.atcc.org/products/PDM-17,Not provided,NaN,Not provided
2,HCM-BROD-0012-C71,HCM-BROD-0012-C71,Not provided,Cell Line,NaN,NaN,Adherent,Stem Cell Technologies - 5751,NeuroCult™ NS-A Proliferation Kit (Human),Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-19,https://www.atcc.org/products/PDM-19,Not provided,NaN,Not provided
3,HCM-BROD-0047-C71,HCM-BROD-0047-C71,Not provided,Cell Line,NaN,NaN,Adherent,Cellaria Bio CM-0001,Renaissance Essential Tumor Medium,Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-23,https://www.atcc.org/products/PDM-23,Not provided,NaN,Not provided
4,HCM-BROD-0029-C71,HCM-BROD-0029-C71,Not provided,"3-D: Other (e.g. neurosphere, air-liquid inter...",NaN,NaN,Mixed: suspension and aggregate,Stem Cell Technologies - 5751,NeuroCult™ NS-A Proliferation Kit (Human),Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-22,https://www.atcc.org/products/PDM-22,Not provided,NaN,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,HCM-BROD-0230-C25,HCM-BROD-0230-C25,Not provided,3-D: Organoid,NaN,NaN,Embedded 3D culture,ATCC ACS-7101,Organoid Growth Kit 1B,Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-171,https://www.atcc.org/products/PDM-171,Not provided,NaN,Not provided
227,HCM-BROD-0019-C25,HCM-BROD-0019-C25,Not provided,3-D: Organoid,NaN,NaN,Embedded 3D culture,ATCC ACS-7101,Organoid Growth Kit 1B,Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-166,https://www.atcc.org/products/PDM-166,Not provided,NaN,Not provided
228,HCM-BROD-0204-C25,HCM-BROD-0204-C25,Not provided,3-D: Organoid,NaN,NaN,Embedded 3D culture,ATCC ACS-7101,Organoid Growth Kit 1B,Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-173,https://www.atcc.org/products/PDM-173,Not provided,NaN,Not provided
229,HCM-BROD-0594-C43,HCM-BROD-0594-C43,Not provided,Cell Line,NaN,NaN,Adherent,Propagenix 256-100,Propagenix Conditioned Medium,Not provided,...,Not provided,Not provided,NaN,ATCC,Commercial,PDM-495,https://www.atcc.org/products/PDM-495,Not provided,NaN,Not provided


In [227]:
mv_sheet = read_metadata_without_fields(join(home, 'BROD', 'BROD_metadata-model_validation.tsv'))
mv_template = read_metadata_with_fields(join(home, 'BROD', 'BROD_metadata-model_validation.tsv')).iloc[0:4]
x = merged[['model_id', 'morphological_features', 'STR_analysis', 'model_purity']]
mv_sheet['model_id'] = merged['model_id']
mv_sheet = mv_sheet.fillna('Not provided')
mv_sheet['morphological_features'] = merged['morphological_features']
mv_sheet['STR_analysis'] = merged['STR_analysis']
mv_sheet['model_purity'] = merged['model_purity']
pd.concat([mv_template, mv_sheet[mv_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'BROD', 'BROD_metadata-model_validation.tsv'), sep='\t', index=False)

In [252]:
patient_sheet = read_metadata_without_fields(join(home, 'BROD', 'BROD_metadata-patient.tsv'))
patient_template = read_metadata_with_fields(join(home, 'BROD', 'BROD_metadata-patient.tsv')).iloc[0:4]
patient_sheet['age_category'] = ['Adult' if int(r) > 21 else 'Paediatric' for r in patient_sheet['age_at_initial_diagnosis']]
pd.concat([patient_template, patient_sheet[patient_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'BROD', 'BROD_metadata-patient.tsv'), sep='\t', index=False)

In [262]:
ps_sheet = read_metadata_without_fields(join(home, 'BROD', 'BROD_metadata-patient_sample.tsv'))
ps_template = read_metadata_with_fields(join(home, 'BROD', 'BROD_metadata-patient_sample.tsv')).iloc[0:4]
ps_sheet['virology_status'] = 'Not detected'
pd.concat([ps_template, ps_sheet[ps_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'BROD', 'BROD_metadata-patient_sample.tsv'), sep='\t', index=False)

In [260]:
merged.columns

Index(['model_id', 'model_name', 'model_name_aliases', 'type', 'parent_id',
       'origin_patient_sample_id', 'growth_properties_og', 'media_id_og',
       'growth_media_og', 'plate_coating', 'other_plate_coating',
       'passage_number', 'contaminated_og', 'contamination_details_og',
       'supplements_og', 'drug', 'drug_concentration', 'publications',
       'supplier', 'supplier_type', 'catalog_number', 'vendor_link', 'rrid',
       'external_ids', 'comments', 'atcc', 'url',
       'product_information.product_category',
       'product_information.product_type', 'product_information.organism',
       'product_information.morphology', 'product_information.tissue',
       'product_information.disease', 'product_information.applications',
       'product_information.product_format',
       'product_information.storage_conditions',
       'general.specific_applications', 'characteristics.cells_per_vial',
       'characteristics.volume', 'characteristics.growth_properties',
       'c

In [261]:
merged[['model_id', 'virology_status']]

,model_id,virology_status
0,HCM-BROD-0011-C71,Not detected
1,HCM-BROD-0003-C71,Not detected
2,HCM-BROD-0012-C71,Not detected
3,HCM-BROD-0047-C71,Not detected
4,HCM-BROD-0029-C71,Not detected
...,...,...
226,HCM-BROD-0230-C25,Not detected
227,HCM-BROD-0019-C25,Not detected
228,HCM-BROD-0204-C25,Not detected
229,HCM-BROD-0594-C43,Not detected


In [266]:
cshl_atcc = fetch_from_atcc(read_metadata_without_fields(join(home, 'CSHL', 'CSHL_metadata-cell_model.tsv')))

IndexError: list index out of range

In [267]:
in_vitro_models = read_metadata_without_fields(join(home, 'CSHL', 'CSHL_metadata-cell_model.tsv'))

In [278]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode if desired
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
first = True
text = pd.DataFrame()
#attributes = list()
for url in in_vitro_models['vendor_link'].str.lower():
    if url == 'not provided':
        continue
    driver.get(url)
    if first:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Allow all cookies']"))).click()
        first = False
    #print('Expand')
    sleep(2)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Expand all')]"))).click()
    headers = list()
    h3 = driver.find_elements(By.XPATH, "//h3")
    append_it = False
    for h in h3:
        if len(headers) == 0:
            headers.append('Product Information')
        if h.text == "General" or h.text == 'Characteristics' or append_it:
            headers.append(h.text)
            append_it = True
        if h.text == "Legal disclaimers":
            append_it = False

    dls = driver.find_elements(By.XPATH, "//dl")
    i = 0
    df = pd.DataFrame()
    df['atcc'] = [url.replace('https://www.atcc.org/products/', '').upper()]
    df['url'] = url
    for dl in dls:
        soup = BeautifulSoup(dl.get_attribute("outerHTML"), 'html.parser')
        dt_elements = soup.find_all('dt', class_='product-information__title')
        dd_elements = soup.find_all('dd', class_='product-information__data')
        dt_values = [headers[i].replace(' ', '_').lower()+'.'+dt.get_text(strip=True).replace(' ', '_').lower() for dt in dt_elements]
        i = i+1
        dd_values = [dd.get_text(strip=True) for dd in dd_elements]
        for j in range(len(dt_values)):
            df[dt_values[j]] = dd_values[j]  
    text = pd.concat([text, df]).reset_index(drop=True)
driver.quit()

In [279]:
text

,atcc,url,product_information.product_category,product_information.product_type,product_information.organism,product_information.morphology,product_information.tissue,product_information.disease,product_information.applications,product_information.product_format,...,quality_control_specifications.mycoplasma_contamination,quality_control_specifications.virus_testing,history.depositors,history.year_of_origin,history.special_collection,legal_disclaimers.intended_use,legal_disclaimers.warranty,legal_disclaimers.disclaimers,characteristics.metastatic,characteristics.oncogene
0,PDM-1,https://www.atcc.org/products/pdm-1,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Cecum,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Cytomegalovirus (CMV): Not detectedHuman papil...,Cold Spring Harbor Laboratory,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
1,PDM-2,https://www.atcc.org/products/pdm-2,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Sigmoid colon,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Cytomegalovirus (CMV): Not detectedHuman papil...,Cold Spring Harbor Laboratory,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
2,PDM-6,https://www.atcc.org/products/pdm-6,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Rectosigmoid junction,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Cytomegalovirus (CMV): Not detectedHuman papil...,Cold Spring Harbor Laboratory,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
3,PDM-7,https://www.atcc.org/products/pdm-7,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Sigmoid colon,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Cytomegalovirus (CMV): Not detectedHuman papil...,Cold Spring Harbor Laboratory,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
4,PDM-5,https://www.atcc.org/products/pdm-5,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Sigmoid colon,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Cytomegalovirus (CMV): Not detectedHuman papil...,Cold Spring Harbor Laboratory,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,PDM-528,https://www.atcc.org/products/pdm-528,Human cells,Cell modelOrganoid,"Homo sapiens, human",Organoid,Uterus,Cancer; Primary,Cancer research3D cell culture,Frozen,...,Not detected,Cytomegalovirus (CMV): Not detectedEpstein-Bar...,Cold Spring Harbor Laboratory,2021,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
94,PDM-440,https://www.atcc.org/products/pdm-440,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Neck,Cancer; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Cytomegalovirus (CMV): Not detectedEpstein-Bar...,Cold Spring Harbor Laboratory,2022,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provi

In [280]:
text['handling_information.complete_medium'].unique()

array(['To prepare the complete medium for this organoid please refer to theOrganoid Media Formulation #1.ATCC offers the recombinant proteins, small molecules and other supplements to make this complete medium; CoreKit 1A (ATCC ACS-7100) provides these supplements in convenient, pre-portioned, ready-to-reconstitute format that does not require aliquoting or storage once prepared.',
       'To prepare the complete medium for this organoid please refer to theOrganoid Media Formulation #3.ATCC offers the recombinant proteins, small molecules and other supplements to make this complete medium; CoreKit 1B (ATCC ACS-7101) provides these supplements in convenient, pre-portioned, ready-to-reconstitute format that does not require aliquoting or storage once prepared.',
       'To prepare the complete medium for this organoid please refer to theOrganoid Media Formulation #2. ATCC offers the recombinant proteins, small molecules and other supplements to make this complete medium; CoreKit 1C (ATC

In [304]:
text[text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #12')]['url'].unique()

array(['https://www.atcc.org/products/pdm-428',
       'https://www.atcc.org/products/pdm-440',
       'https://www.atcc.org/products/pdm-431'], dtype=object)

In [307]:
conditions = [text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #3'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #6'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #8'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #11'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #12'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #2'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #1\.')]


replacements = ['ATCC ACS-7101', 'ATCC ACS-7106', 'ATCC ACS-7105', 'ATCC Organoid media 11a', 'ATCC Organoid media 12a', 'ATCC ACS-7102', 'ATCC ACS-7100']


growth_media = ['Organoid Growth Kit 1B', 'Organoid Growth Kit 1G', 'Organoid Growth Kit 1F', 'ATCC Organoid media 11a', 'ATCC Organoid media 12a', 'Organoid Growth Kit 1C', 'Organoid Growth Kit 1A']

supplements = ['Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Not provided']

text['media_id'] = np.select(conditions, replacements, default='Not provided')
text['growth_media'] = np.select(conditions, growth_media, default='Not provided')
text['supplements'] = np.select(conditions, supplements, default='Not provided')
text['growth_properties'] = text['characteristics.growth_properties']
text['contaminated'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'No', 'Yes')

text['contamination_details'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'No contamination detected', 'Not provided')

text['model_purity'] = np.where(text['characteristics.metastatic'].fillna('Not provided') == 'Not provided', 'Not provided', 'Tumor cells')

text['STR_analysis'] = 'Not provided'#np.where(text['quality_control_specifications.str_profiling'].fillna('Not provided') == 'Not provided', 'No', 'Yes')

text['morphological_features'] = text['product_information.morphology'].fillna('Not provided')

text['virology_status'] = np.where(text['quality_control_specifications.virus_testing'].str.lower().str.replace('detected', 'detected, ').str.replace('not detected', '').str.contains('detected'), 'Detected', 'Not detected')

In [314]:
text[np.where(text['quality_control_specifications.virus_testing'].str.lower().str.replace('detected', 'detected, ').str.replace('not detected', '').str.contains('detected'), True, False)]

,atcc,url,product_information.product_category,product_information.product_type,product_information.organism,product_information.morphology,product_information.tissue,product_information.disease,product_information.applications,product_information.product_format,...,media_id,growth_media,supplements,growth_properties,contaminated,contamination_details,model_purity,STR_analysis,morphological_features,virology_status
54,PDM-29,https://www.atcc.org/products/pdm-29,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Pancreas,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7101,Organoid Growth Kit 1B,Not provided,Embedded 3D culture,Yes,No contamination detected,Not provided,Not provided,organoid,Detected
78,PDM-198,https://www.atcc.org/products/pdm-198,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Pancreas,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7101,Organoid Growth Kit 1B,Not provided,Embedded 3D culture,Yes,No contamination detected,Not provided,Not provided,organoid,Detected


In [309]:
cell_sheet = read_metadata_without_fields(join(home, 'CSHL', 'CSHL_metadata-cell_model.tsv'))
cell_template = read_metadata_with_fields(join(home, 'CSHL', 'CSHL_metadata-cell_model.tsv')).iloc[0:4]
merged = cell_sheet.merge(text, left_on='catalog_number', right_on='atcc', how='left', suffixes=('_og', '_supplier'))
merged['growth_properties'] = merged['growth_properties_supplier']
merged['media_id'] = merged['media_id_supplier']
merged['growth_media'] = merged['growth_media_supplier']
merged['contaminated'] = merged['contaminated_supplier']
merged['contamination_details'] = merged['contamination_details_supplier']
merged['supplements'] = merged['supplements_supplier']
pd.concat([cell_template, merged[cell_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'CSHL', 'CSHL_metadata-cell_model.tsv'), sep='\t', index=False)

In [310]:
mv_sheet = read_metadata_without_fields(join(home, 'CSHL', 'CSHL_metadata-model_validation.tsv'))
mv_template = read_metadata_with_fields(join(home, 'CSHL', 'CSHL_metadata-model_validation.tsv')).iloc[0:4]
x = merged[['model_id', 'morphological_features', 'STR_analysis', 'model_purity']]
mv_sheet['model_id'] = merged['model_id']
mv_sheet = mv_sheet.fillna('Not provided')
mv_sheet['morphological_features'] = merged['morphological_features']
mv_sheet['STR_analysis'] = merged['STR_analysis']
mv_sheet['model_purity'] = merged['model_purity']
pd.concat([mv_template, mv_sheet[mv_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'CSHL', 'CSHL_metadata-model_validation.tsv'), sep='\t', index=False)

In [312]:
patient_sheet = read_metadata_without_fields(join(home, 'CSHL', 'CSHL_metadata-patient.tsv'))
patient_template = read_metadata_with_fields(join(home, 'CSHL', 'CSHL_metadata-patient.tsv')).iloc[0:4]
patient_sheet['age_category'] = ['Not provided' if r.__contains__('Not') else 'Adult' if int(r) > 21 else 'Paediatric' for r in patient_sheet['age_at_initial_diagnosis']]
pd.concat([patient_template, patient_sheet[patient_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'CSHL', 'CSHL_metadata-patient.tsv'), sep='\t', index=False)

In [ ]:
ps_sheet = read_metadata_without_fields(join(home, 'CSHL', 'CSHL_metadata-patient_sample.tsv'))
ps_template = read_metadata_with_fields(join(home, 'CSHL', 'CSHL_metadata-patient_sample.tsv')).iloc[0:4]
ps_sheet['virology_status'] = 'Not detected'
pd.concat([ps_template, ps_sheet[ps_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'CSHL', 'CSHL_metadata-patient_sample.tsv'), sep='\t', index=False)

In [317]:
sang_atcc = fetch_from_atcc(read_metadata_without_fields(join(home, 'SANG', 'SANG_metadata-cell_model.tsv')))

In [327]:
text = sang_atcc
text['handling_information.complete_medium'].unique()

,atcc,url,product_information.product_category,product_information.product_type,product_information.organism,product_information.morphology,product_information.tissue,product_information.disease,product_information.applications,product_information.product_format,...,quality_control_specifications.mycoplasma_contamination,quality_control_specifications.virus_testing,history.depositors,history.year_of_origin,history.special_collection,legal_disclaimers.intended_use,legal_disclaimers.warranty,legal_disclaimers.disclaimers,handling_information.handling_notes,characteristics.metastatic
0,PDM-44,https://www.atcc.org/products/pdm-44,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Rectum,Adenoma; Pre-malignant,3D cell cultureCancer research,Frozen,...,Not detected,Hepatitis B virus (HBV): Not detectedCytomegal...,Wellcome Sanger Institute,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
1,PDM-43,https://www.atcc.org/products/pdm-43,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Rectum,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Hepatitis B virus (HBV): Not detectedCytomegal...,Wellcome Sanger Institute,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
2,PDM-67,https://www.atcc.org/products/pdm-67,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Esophagus,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Hepatitis B virus (HBV): Not detectedCytomegal...,Wellcome Sanger Institute,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
3,PDM-71,https://www.atcc.org/products/pdm-71,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Esophagus,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Hepatitis B virus (HBV): Not detectedCytomegal...,Wellcome Sanger Institute,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,Seeding density:We recommend seeding this mode...,NaN
4,PDM-47,https://www.atcc.org/products/pdm-47,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Rectum,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Hepatitis B virus (HBV): Not detectedCytomegal...,Wellcome Sanger Institute,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
5,PDM-46,https://www.atcc.org/products/pdm-46,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Colon,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Hepatitis B virus (HBV): Not detectedCytomegal...,Wellcome Sanger Institute,NaN,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,NaN
6,PDM-90,https://www.atcc.org/products/pdm-90,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Pancreas,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,Not detected,Hepatitis B virus (HBV): Not detectedCytomegal...,Wellcome Sanger Institute,2017,Human Cancer Models Initiative (HCMI),This product is intended for laboratory resear...,The product is provided 'AS IS' and the viabil...,This product is intended for laboratory resear...,NaN,

In [326]:
text[text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #1')]['handling_information.complete_medium'].unique()[0]

'To prepare the complete medium for this organoid please refer to theOrganoid Media Formulation #1.ATCC offers the recombinant proteins, small molecules and other supplements to make this complete medium; CoreKit 1A (ATCC ACS-7100) provides these supplements in convenient, pre-portioned, ready-to-reconstitute format that does not require aliquoting or storage once prepared.'

In [328]:
conditions = [text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #4'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #5'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #7'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #1'),
              ]


replacements = ['ATCC ACS-7103', 'ATCC ACS-7104', 'ATCC ACS-7101', 'ATCC ACS-7100',]


growth_media = ['Organoid Growth Kit 1D', 'Organoid Growth Kit 1E', 'Organoid Growth Kit 1B', 'Organoid Growth Kit 1A',]

supplements = ['Not provided', 'Not provided', 'Not provided', 'Not provided']

text['media_id'] = np.select(conditions, replacements, default='Not provided')
text['growth_media'] = np.select(conditions, growth_media, default='Not provided')
text['supplements'] = np.select(conditions, supplements, default='Not provided')
text['growth_properties'] = text['characteristics.growth_properties']
text['contaminated'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'No', 'Yes')

text['contamination_details'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'No contamination detected', 'Not provided')

text['model_purity'] = np.where(text['characteristics.metastatic'].fillna('Not provided') == 'Not provided', 'Not provided', 'Tumor cells')

text['STR_analysis'] = 'Not provided'#np.where(text['quality_control_specifications.str_profiling'].fillna('Not provided') == 'Not provided', 'No', 'Yes')

text['morphological_features'] = text['product_information.morphology'].fillna('Not provided')

text['virology_status'] = np.where(text['quality_control_specifications.virus_testing'].str.lower().str.replace('detected', 'detected, ').str.replace('not detected', '').str.contains('detected'), 'Detected', 'Not detected')

In [330]:
text['growth_media'].unique()

array(['Organoid Growth Kit 1D', 'Organoid Growth Kit 1E',
       'Organoid Growth Kit 1B', 'Organoid Growth Kit 1A'], dtype=object)

In [331]:
cell_sheet = read_metadata_without_fields(join(home, 'SANG', 'SANG_metadata-cell_model.tsv'))
cell_template = read_metadata_with_fields(join(home, 'SANG', 'SANG_metadata-cell_model.tsv')).iloc[0:4]
merged = cell_sheet.merge(text, left_on='catalog_number', right_on='atcc', how='left', suffixes=('_og', '_supplier'))
merged['growth_properties'] = merged['growth_properties_supplier']
merged['media_id'] = merged['media_id_supplier']
merged['growth_media'] = merged['growth_media_supplier']
merged['contaminated'] = merged['contaminated_supplier']
merged['contamination_details'] = merged['contamination_details_supplier']
merged['supplements'] = merged['supplements_supplier']
pd.concat([cell_template, merged[cell_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'SANG', 'SANG_metadata-cell_model.tsv'), sep='\t', index=False)

In [332]:
mv_sheet = read_metadata_without_fields(join(home, 'SANG', 'SANG_metadata-model_validation.tsv'))
mv_template = read_metadata_with_fields(join(home, 'SANG', 'SANG_metadata-model_validation.tsv')).iloc[0:4]
x = merged[['model_id', 'morphological_features', 'STR_analysis', 'model_purity']]
mv_sheet['model_id'] = merged['model_id']
mv_sheet = mv_sheet.fillna('Not provided')
mv_sheet['morphological_features'] = merged['morphological_features']
mv_sheet['STR_analysis'] = merged['STR_analysis']
mv_sheet['model_purity'] = merged['model_purity']
pd.concat([mv_template, mv_sheet[mv_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'SANG', 'SANG_metadata-model_validation.tsv'), sep='\t', index=False)

In [333]:
patient_sheet = read_metadata_without_fields(join(home, 'SANG', 'SANG_metadata-patient.tsv'))
patient_template = read_metadata_with_fields(join(home, 'SANG', 'SANG_metadata-patient.tsv')).iloc[0:4]
patient_sheet['age_category'] = ['Not provided' if r.__contains__('Not') else 'Adult' if int(r) > 21 else 'Paediatric' for r in patient_sheet['age_at_initial_diagnosis']]
pd.concat([patient_template, patient_sheet[patient_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'SANG', 'SANG_metadata-patient.tsv'), sep='\t', index=False)

In [334]:
text

,atcc,url,product_information.product_category,product_information.product_type,product_information.organism,product_information.morphology,product_information.tissue,product_information.disease,product_information.applications,product_information.product_format,...,media_id,growth_media,supplements,growth_properties,contaminated,contamination_details,model_purity,STR_analysis,morphological_features,virology_status
0,PDM-44,https://www.atcc.org/products/pdm-44,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Rectum,Adenoma; Pre-malignant,3D cell cultureCancer research,Frozen,...,ATCC ACS-7103,Organoid Growth Kit 1D,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
1,PDM-43,https://www.atcc.org/products/pdm-43,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Rectum,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7103,Organoid Growth Kit 1D,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
2,PDM-67,https://www.atcc.org/products/pdm-67,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Esophagus,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7104,Organoid Growth Kit 1E,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
3,PDM-71,https://www.atcc.org/products/pdm-71,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Esophagus,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7104,Organoid Growth Kit 1E,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
4,PDM-47,https://www.atcc.org/products/pdm-47,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Rectum,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7103,Organoid Growth Kit 1D,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
5,PDM-46,https://www.atcc.org/products/pdm-46,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Colon,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7103,Organoid Growth Kit 1D,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
6,PDM-90,https://www.atcc.org/products/pdm-90,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Pancreas,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7101,Organoid Growth Kit 1B,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
7,PDM-58,https://www.atcc.org/products/pdm-58,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Colon,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7103,Organoid Growth Kit 1D,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
8,PDM-73,https://www.atcc.org/products/pdm-73,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Esophagus,Malignant neoplasm,3D cell cultureCancer research,Frozen,...,ATCC ACS-7104,Organoid Growth Kit 1E,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
9,PDM-79,https://www.atcc.org/products/pdm-79,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Esophagus,Adenocarcinoma; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7104,Organoid Growth Kit 1E,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected


In [335]:
wcmc_atcc = fetch_from_atcc(read_metadata_without_fields(join(home, 'WCMC', 'WCMC_metadata-cell_model.tsv')))

In [341]:
text = wcmc_atcc
text['handling_information.complete_medium'] = text['handling_information.complete_medium'].fillna('Not provided')

In [348]:
text

,atcc,url,product_information.product_category,product_information.product_type,product_information.organism,product_information.morphology,product_information.tissue,product_information.disease,product_information.applications,product_information.product_format,...,media_id,growth_media,supplements,growth_properties,contaminated,contamination_details,model_purity,STR_analysis,morphological_features,virology_status
0,PDM-513,https://www.atcc.org/products/pdm-513,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Large intestine; Colon,Cancer; Primary,Cancer research3D cell culture,Frozen,...,ATCC ACS-7105,Organoid Growth Kit 1F,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
1,PDM-515,https://www.atcc.org/products/pdm-515,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Ovary,Cancer; Primary,Cancer research3D cell culture,Frozen,...,ATCC Organoid Media #10,Organoid Media Formulation #10,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
2,PDM-514,https://www.atcc.org/products/pdm-514,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Ovary,Cancer; Primary,Cancer research3D cell culture,Frozen,...,ATCC Organoid Media #10,Organoid Media Formulation #10,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
3,PDM-375,https://www.atcc.org/products/pdm-375,Human cells,Cell modelOrganoid,"Homo sapiens, human",Organoid,Large intestine; Colon,Cancer; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7105,Organoid Growth Kit 1F,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,Organoid,Not detected
4,PDM-683,https://www.atcc.org/products/pdm-683,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Lung,Cancer; Primary,3D cell cultureCancer research,Frozen,...,ATCC Organoid Media #10,Organoid Media Formulation #10,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
5,PDM-685,https://www.atcc.org/products/pdm-685,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Lung,Cancer; Primary,3D cell cultureCancer research,Frozen,...,ATCC Organoid Media #10,Organoid Media Formulation #10,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected
6,PDM-376,https://www.atcc.org/products/pdm-376,Human cells,OrganoidCell model,"Homo sapiens, human",Organoid,Stomach,Cancer; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7105,Organoid Growth Kit 1F,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,Organoid,Not detected
7,PDM-512,https://www.atcc.org/products/pdm-512,Human cells,Cell modelOrganoid,"Homo sapiens, human",Organoid,Esophagus,Cancer; Primary,3D cell cultureCancer research,Frozen,...,ATCC ACS-7105,Organoid Growth Kit 1F,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,Organoid,Not detected
8,PDM-519,https://www.atcc.org/products/pdm-519,Human cells,Cell modelOrganoid,"Homo sapiens, human",Organoid,Stomach,NaN,3D cell cultureCancer research,Frozen,...,ATCC ACS-7105,Organoid Growth Kit 1F,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,Organoid,Not detected
9,PDM-377,https://www.atcc.org/products/pdm-377,Human cells,Cell modelOrganoid,"Homo sapiens, human",organoid,Stomach,Cancer; Primary,3D cell cultureCancer research,Frozen,...,Not provided,Not provided,Not provided,Embedded 3D culture,No,No contamination detected,Not provided,Not provided,organoid,Not detected


In [343]:
conditions = [text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #9'),
              text['handling_information.complete_medium'].str.contains('theOrganoid Media Formulation #10'),
              ]


replacements = ['ATCC ACS-7105', 'ATCC Organoid Media #10']


growth_media = ['Organoid Growth Kit 1F', 'Organoid Media Formulation #10']

supplements = ['Not provided', 'Not provided']

text['media_id'] = np.select(conditions, replacements, default='Not provided')
text['growth_media'] = np.select(conditions, growth_media, default='Not provided')
text['supplements'] = np.select(conditions, supplements, default='Not provided')
text['growth_properties'] = text['characteristics.growth_properties']
text['contaminated'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'No', 'Yes')

text['contamination_details'] = np.where((text['quality_control_specifications.bacterial_and_fungal_testing'] == 'Not detected') & (text['quality_control_specifications.mycoplasma_contamination'] == 'Not detected'), 'No contamination detected', 'Not provided')

text['model_purity'] = 'Not provided'#np.where(text['characteristics.metastatic'].fillna('Not provided') == 'Not provided', 'Not provided', 'Tumor cells')

text['STR_analysis'] = 'Not provided'#np.where(text['quality_control_specifications.str_profiling'].fillna('Not provided') == 'Not provided', 'No', 'Yes')

text['morphological_features'] = text['product_information.morphology'].fillna('Not provided')

text['virology_status'] = np.where(text['quality_control_specifications.virus_testing'].str.lower().str.replace('detected', 'detected, ').str.replace('not detected', '').str.contains('detected'), 'Detected', 'Not detected')

In [344]:
text['growth_media'].unique()

array(['Organoid Growth Kit 1F', 'Organoid Media Formulation #10',
       'Not provided'], dtype=object)

In [345]:
cell_sheet = read_metadata_without_fields(join(home, 'WCMC', 'WCMC_metadata-cell_model.tsv'))
cell_template = read_metadata_with_fields(join(home, 'WCMC', 'WCMC_metadata-cell_model.tsv')).iloc[0:4]
merged = cell_sheet.merge(text, left_on='catalog_number', right_on='atcc', how='left', suffixes=('_og', '_supplier'))
merged['growth_properties'] = merged['growth_properties_supplier']
merged['media_id'] = merged['media_id_supplier']
merged['growth_media'] = merged['growth_media_supplier']
merged['contaminated'] = merged['contaminated_supplier']
merged['contamination_details'] = merged['contamination_details_supplier']
merged['supplements'] = merged['supplements_supplier']
pd.concat([cell_template, merged[cell_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'WCMC', 'WCMC_metadata-cell_model.tsv'), sep='\t', index=False)

In [347]:
mv_sheet = read_metadata_without_fields(join(home, 'WCMC', 'WCMC_metadata-model_validation.tsv'))
mv_template = read_metadata_with_fields(join(home, 'WCMC', 'WCMC_metadata-model_validation.tsv')).iloc[0:4]
x = merged[['model_id', 'morphological_features', 'STR_analysis', 'model_purity']]
mv_sheet['model_id'] = merged['model_id']
mv_sheet = mv_sheet.fillna('Not provided')
mv_sheet['morphological_features'] = merged['morphological_features']
mv_sheet['STR_analysis'] = merged['STR_analysis']
mv_sheet['model_purity'] = merged['model_purity']
pd.concat([mv_template, mv_sheet[mv_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'WCMC', 'WCMC_metadata-model_validation.tsv'), sep='\t', index=False)

In [349]:
patient_sheet = read_metadata_without_fields(join(home, 'WCMC', 'WCMC_metadata-patient.tsv'))
patient_template = read_metadata_with_fields(join(home, 'WCMC', 'WCMC_metadata-patient.tsv')).iloc[0:4]
patient_sheet['age_category'] = ['Not provided' if r.__contains__('Not') else 'Adult' if int(r) > 21 else 'Paediatric' for r in patient_sheet['age_at_initial_diagnosis']]
pd.concat([patient_template, patient_sheet[patient_sheet.columns]]).reset_index(drop=True).to_csv(join(home, 'WCMC', 'WCMC_metadata-patient.tsv'), sep='\t', index=False)

In [ ]:
supplier_dict = {'': '',
                 'ATCC':'https://www.atcc.org/products/',
                 'AddexBio': f'https://www.cellosaurus.org/{RRID}',
                 'BioIVT': '',
                 'CLS': f'https://www.cellosaurus.org/{RRID}',
                 'CellBank Australia': f'https://www.cellosaurus.org/{RRID}',
                 'Childhood Cancer Repository': '',
                 'DSMZ': f'https://www.dsmz.de/catalogues/details/culture/{model_number}.html',
                 'ECACC': f'https://www.phe-culturecollections.org.uk/products/celllines/generalcell/detail.jsp?refId={model_number}&collection=ecacc_gc',
                 'ECACC;DSMZ': f'https://www.dsmz.de/catalogues/details/culture/{model_number}.html',
                 'Horizon Discovery': '',
                 'ICLC': '',
                 'JCRB': f'https://cellbank.nibiohn.go.jp/~cellbank/en/search_res_det.cgi?RNO={model_number}',
                 'KCLB': f'https://cellbank.snu.ac.kr/main/tmpl/sub_main.php?m_cd=6&m_id=0201&strQ={model_number}',
                 'Lonza': '',
                 'Mesobank': '',
                 'NCI': f'https://www.cellosaurus.org/{RRID}',
                 'NCI-Navy Medical Oncology Branch': '',
                 'RIKEN': f'https://cellbank.brc.riken.jp/cell_bank/CellInfo/?cellNo={model_number}&lang=En',
                 'Rockland': '',
                 'TKG': '',
                 'Ximbio': '',
                 'unknown': ''}

In [469]:
cmp_sheet = read_metadata_without_fields(join(home, 'CMP', 'CMP_metadata-cell_model.tsv'))
cmp_template = read_metadata_with_fields(join(home, 'CMP', 'CMP_metadata-cell_model.tsv')).iloc[0:4]
cmp_sheet[['supplier', 'catalog_number']] = cmp_sheet['supplier'].fillna('Not provided').str.replace('ATCC:CR:', 'ATCC:CR').str.replace('ECACC:96090513;DSMZ:ACC-558', 'ECACC;DSMZ:96090513;ACC-558').str.split(':', expand=True).fillna('Not provided')
cmp_sheet['supplier_type'] = ['Not provided' if r.lower() == 'not provided' else 'Commercial' for r in cmp_sheet['supplier']]
cmp_sheet['rrid'] = cmp_sheet['external_ids'].str.extract(r'(CVCL_.{4})').fillna('Not provided')
cmp_sheet['STR_analysis'] = ''
#cmp_sheet['cell_line_provider_url'] = list()
for i in tqdm(range(cmp_sheet.shape[0])):
    rrid = cmp_sheet['rrid'][i]
    supplier_name = cmp_sheet['supplier'][i]
    catalog_number = cmp_sheet['catalog_number'][i]
    if rrid == 'Not provided':
        continue
    cellosaurus = f'https://www.cellosaurus.org/{rrid}'
    response = get(cellosaurus)
    if response.status_code == 200: 
        soup = BeautifulSoup(response.text, 'html.parser')
        publications = soup.find('th', text='Publications')
        if publications is not None:
            publications = publications.find_next('td')
            pmids = ''
            if publications is not None:
                for pubmed in publications.find_all('a'):
                    if pubmed['href'].__contains__('pubmed'):
                        pmids += 'PMID: ' + pubmed.text + ','
                pmids = pmids[:-1]
                if cmp_sheet.loc[i, 'publications'] is not str:
                    cmp_sheet.loc[i, 'publications'] = pmids
                else:
                    cmp_sheet.loc[i, 'publications'] += ',' + pmids
                pmid_list = cmp_sheet.loc[i, 'publications'].split(',')
                unique_pmids = set(pmid_list)
                cmp_sheet.loc[i, 'publications'] = ','.join(unique_pmids)    
                
        str_profile = soup.find('th', text='STR profile')
        if str_profile is not None:
            str_profile = str_profile.find_next('table')
            result = ''
            for row in str_profile.find_all('tr'):
                j = 0
                for cell in row.find_all('td'):
                    result += cell.get_text()
                    if j == 0:
                        result += ': '
                        j+=1
                result += '; '
            if len(result) > 0:
                result = result[:-2]
                if cmp_sheet.loc[i, 'comments'] is not str:
                    cmp_sheet.loc[i, 'comments'] = 'STR Profile: ' + result
                else:
                    cmp_sheet.loc[i, 'comments'] += ', STR Profile: ' + result
                cmp_sheet.loc[i, 'STR_analysis'] = 'Yes' 
        supplier_section = soup.find('th', text='Cell line collections (Providers)')
        links = None
        if supplier_section:
            values_and_links = supplier_section.find_next('td')
            links = [link['href'] for link in values_and_links.find_all('a', href=True) if len(link['href']) > 0]
            if len(links) == 0:
                links = None
        if links is None:
            if supplier_name == "Childhood Cancer Repository" or supplier_name == "BioIVT":
                values_and_links = soup.find('th', text='Web pages').find_next('td')
                links = [link['href'] for link in values_and_links.find_all('a', href=True) if len(link['href']) > 0]
            if supplier_name == "DSMZ" or supplier_name == "ECACC;DSMZ":
                links = [f'https://www.dsmz.de/collection/catalogue/details/culture/{catalog_number}']
            if supplier_name == "ECACC":
                links = [f'https://www.phe-culturecollections.org.uk/products/celllines/generalcell/detail.jsp?refId={catalog_number}1&collection=ecacc_gc']
        if links is None:
            links = ['Not provided']
        cmp_sheet.loc[i, 'cell_line_provider_url'] = links[0]
cmp_sheet['vendor_link'] = cmp_sheet['cell_line_provider_url'].fillna('Not provided')

  0%|          | 0/1932 [00:00<?, ?it/s]/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_35478/3579978440.py:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  publications = soup.find('th', text='Publications')
/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_35478/3579978440.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  str_profile = soup.find('th', text='STR profile')
/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_35478/3579978440.py:54: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  supplier_section = soup.find('th', text='Cell line collections (Providers)')
  0%|          | 1/1932 [00:01<37:56,  1.18s/it]/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_35478/3579978440.py:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' inst

In [470]:
cmp_sheet

,model_id,model_name,model_name_aliases,type,parent_id,origin_patient_sample_id,growth_properties,media_id,growth_media,plate_coating,...,publications,supplier,supplier_type,catalog_number,vendor_link,rrid,external_ids,comments,STR_analysis,cell_line_provider_url
0,SIDM01774,PK-59,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 25877200,PMID: 22460905,P...",RIKEN,Commercial,RCB1901,Not provided,CVCL_4897,"ACH-000205,CVCL_4897,CCLE_Name:PK59_PANCREAS",STR Profile: Amelogenin: X; CSF1PO: 10; D3S135...,Yes,http://cellbank.brc.riken.jp/cell_bank/CellInf...
1,SIDM00192,SNU-1033,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 10674020,PMID: 19956504,P...",KCLB,Commercial,01033,Not provided,CVCL_5002,"ACH-000286,CVCL_5002,CCLE_Name:SNU1033_LARGE_I...",STR Profile: Amelogenin: X; CSF1PO: 10; D1S165...,Yes,https://cellbank.snu.ac.kr/main/tmpl/sub_main....
2,SIDM01447,SNU-466,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 22460905,PMID: 19956504,P...",KCLB,Commercial,00466,Not provided,CVCL_5064,"ACH-000289,CVCL_5064,CCLE_Name:SNU466_CENTRAL_...","STR Profile: Amelogenin: X,Y; CSF1PO: 12; D3S1...",Yes,https://cellbank.snu.ac.kr/main/tmpl/sub_main....
3,SIDM01554,IST-MES-2,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 22460905,PMID: 10232421,P...",ICLC,Commercial,HTL01007,Not provided,CVCL_1312,"ACH-000331,CVCL_1312,CCLE_Name:ISTMES2_PLEURA","STR Profile: Amelogenin: X,Y; CSF1PO: 10,11; D...",Yes,http://www.iclc.it/details/det_list.php?line_i...
4,SIDM01689,MUTZ-5,Not provided,Cell Line,NaN,NaN,Suspension,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 22460905,PMID: 30285677,P...",DSMZ,Commercial,ACC-490,Not provided,CVCL_1873,"ACH-000492,CVCL_1873,CCLE_Name:MUTZ5_HAEMATOPO...","STR Profile: Amelogenin: X; CSF1PO: 8,9; D2S13...",Yes,https://www.dsmz.de/collection/catalogue/detai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,SIDM02153,MM485,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,PMID: 3127479,ECACC,Commercial,11072803,Not provided,Not provided,"COSMIC1206009,ACH-001973",NaN,,NaN
1928,SIDM02155,NZM3,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,NaN,ECACC,Commercial,17020204,Not provided,Not provided,"COSMIC1660299,ACH-001982",NaN,,NaN
1929,SIDM02157,WM3772F,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,NaN,Rockland,Commercial,WM3772F-01-0001,Not provided,Not provided,ACH-002687,NaN,,NaN
1930,SIDM02159,HAP1,Not provided,Cell Line,SIDM02121,SIDS02033,Adherent,Not provided,Not provided,Not provided,...,NaN,Horizon Discovery,Commercial,C631,Not provided,Not provided,ACH-002475,HAP1 is a subclone of KBM7 which is haploid ex...,,NaN


In [472]:
cmp_sheet[cmp_sheet['cell_line_provider_url'].fillna('Not provided') == 'Not provided']

,model_id,model_name,model_name_aliases,type,parent_id,origin_patient_sample_id,growth_properties,media_id,growth_media,plate_coating,...,publications,supplier,supplier_type,catalog_number,vendor_link,rrid,external_ids,comments,STR_analysis,cell_line_provider_url
13,SIDM01387,NCI-H1339,Not provided,Cell Line,NaN,NaN,Not provided,Not provided,Not provided,Not provided,...,"PMID: 19472407,PMID: 25877200,PMID: 1312696,PM...",Not provided,Not provided,Not provided,Not provided,CVCL_A472,"ACH-000921,CVCL_A472,CCLE_Name:NCIH1339_LUNG",STR Profile: Amelogenin: X; CSF1PO: 12; D2S133...,Yes,Not provided
24,SIDM01752,L3-3,Not provided,Cell Line,SIDM01654,SIDS01608,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 25984343,PMID: 22460905,P...",Not provided,Not provided,Not provided,Not provided,CVCL_8147,"ACH-000685,CVCL_8147,CCLE_Name:L33_PANCREAS",Derived through sequential passaging in nude m...,,Not provided
30,SIDM00128,JHU-011,Not provided,Cell Line,NaN,NaN,Not provided,Not provided,Not provided,Not provided,...,"PMID: 21868764,PMID: 35839778,PMID: 12925955,P...",Not provided,Not provided,Not provided,Not provided,CVCL_5986,"COSMIC1240161,ACH-002249,CVCL_5986,CCLE_Name:J...","STR Profile: Amelogenin: X; CSF1PO: 10,12; D5S...",Yes,Not provided
39,SIDM01287,HCM-SANG-0290-C15,Not provided,Organoid,NaN,NaN,Not provided,Not provided,Not provided,Not provided,...,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,NaN,,NaN
54,SIDM00205,EW-1,Not provided,Cell Line,NaN,NaN,Not provided,Not provided,Not provided,Not provided,...,"PMID: 6713356,PMID: 35839778,PMID: 11423975,PM...",Not provided,Not provided,Not provided,Not provided,CVCL_1208,"COSMIC949163,ACH-002110,CVCL_1208,CCLE_Name:EW...","STR Profile: Amelogenin: X,Y; CSF1PO: 11; D5S8...",Yes,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,SIDM02153,MM485,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,PMID: 3127479,ECACC,Commercial,11072803,Not provided,Not provided,"COSMIC1206009,ACH-001973",NaN,,NaN
1928,SIDM02155,NZM3,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,NaN,ECACC,Commercial,17020204,Not provided,Not provided,"COSMIC1660299,ACH-001982",NaN,,NaN
1929,SIDM02157,WM3772F,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,NaN,Rockland,Commercial,WM3772F-01-0001,Not provided,Not provided,ACH-002687,NaN,,NaN
1930,SIDM02159,HAP1,Not provided,Cell Line,SIDM02121,SIDS02033,Adherent,Not provided,Not provided,Not provided,...,NaN,Horizon Discovery,Commercial,C631,Not provided,Not provided,ACH-002475,HAP1 is a subclone of KBM7 which is haploid ex...,,NaN


In [776]:
cmp_sheet['vendor_link'].str.split('://', expand=True)[1].str.split('/',expand=True)[0].unique()

array(['cellbank.brc.riken.jp', 'cellbank.snu.ac.kr', 'www.iclc.it',
       'www.dsmz.de', 'bcrj.org.br', 'www.addexbio.com',
       'cellbank.nibiohn.go.jp', None, 'www.atcc.org', 'www.rockland.com',
       'www.cctcc.org', 'dtp.cancer.gov',
       'www.phe-culturecollections.org.uk', 'cls.shop',
       'www.cancertools.org', 'www.abcam.com', 'ximbio.com',
       'www.merckmillipore.com', 'catalog.coriell.org', 'www.fli.de',
       'www.ibvr.org', 'catalog.bcrc.firdi.org.tw',
       'www.cellbankaustralia.com', 'www.cccells.org',
       'www.cellosaurus.org', 'bioivt.com', 'www.kmcellbank.com',
       'www.ubigene.us'], dtype=object)

In [1286]:
cmp_sheet[cmp_sheet['vendor_link'].str.contains('dsmz')]

,model_id,model_name,model_name_aliases,type,parent_id,origin_patient_sample_id,growth_properties,media_id,growth_media,plate_coating,...,publications,supplier,supplier_type,catalog_number,vendor_link,rrid,external_ids,comments,STR_analysis,cell_line_provider_url
4,SIDM01689,MUTZ-5,Not provided,Cell Line,NaN,NaN,Suspension,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 22460905,PMID: 30285677,P...",DSMZ,Commercial,ACC-490,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1873,"ACH-000492,CVCL_1873,CCLE_Name:MUTZ5_HAEMATOPO...","STR Profile: Amelogenin: X; CSF1PO: 8,9; D2S13...",Yes,https://www.dsmz.de/collection/catalogue/detai...
8,SIDM01587,JMSU-1,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 7483139,PMID: 26589293,PMID: 22460905,PM...",DSMZ,Commercial,ACC-505,https://www.dsmz.de/collection/catalogue/detai...,CVCL_2081,"ACH-000753,CVCL_2081,CCLE_Name:JMSU1_URINARY_T...",STR Profile: Amelogenin: X; CSF1PO: 11; D2S133...,Yes,https://www.dsmz.de/collection/catalogue/detai...
11,SIDM00074,BL-70,Not provided,Cell Line,NaN,NaN,Not provided,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 3905596,PMID: 25355872,PM...",DSMZ,Commercial,ACC-233,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1088,"COSMIC910707,ACH-000402,CVCL_1088,CCLE_Name:BL...","STR Profile: Amelogenin: X,Y; CSF1PO: 11; D2S1...",Yes,https://www.dsmz.de/collection/catalogue/detai...
16,SIDM00993,AMO-1,Not provided,Cell Line,NaN,NaN,Suspension,Not provided,Not provided,Not provided,...,"PMID: 10936422,PMID: 26589293,PMID: 9738977,PM...",DSMZ,Commercial,ACC-538,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1806,"COSMIC1295741,ACH-000838,CVCL_1806,CCLE_Name:A...",STR Profile: Amelogenin: X; CSF1PO: 12; D2S133...,Yes,https://www.dsmz.de/collection/catalogue/detai...
18,SIDM00990,BEN,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 25877200,PMID: 35839778,P...",DSMZ,Commercial,ACC-254,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1082,"COSMIC753534,ACH-000603,CVCL_1082,CCLE_Name:BE...","STR Profile: Amelogenin: X (DSMZ) X,Y (Cosmic...",Yes,https://www.dsmz.de/collection/catalogue/detai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,SIDM01785,NGP,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 15892104,PMID: 28350380,PMID: 16822308,P...",DSMZ,Commercial,ACC-676,https://www.dsmz.de/collection/catalogue/detai...,CVCL_2141,"ACH-001366,CVCL_2141,CCLE_Name:NGP_AUTONOMIC_G...","STR Profile: Amelogenin: X,Y; CSF1PO: 11,12; D...",Yes,https://www.dsmz.de/collection/catalogue/detai...
1862,SIDM01061,HN,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 25877200,PMID: 22460905,P...",DSMZ,Commercial,ACC-417,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1283,"COSMIC907059,ACH-001090,CVCL_1283,CCLE_Name:HN...",STR Profile: Amelogenin: X; CSF1PO: 11; D2S133...,Yes,https://www.dsmz.de/collection/catalogue/detai...
1864,SIDM00985,BL-41,Not provided,Cell Line,NaN,NaN,Suspension,Not provided,Not provided,Not provided,...,"PMID: 2645221,PMID: 26589293,PMID: 3905596,PMI...",DSMZ,Commercial,ACC-160,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1087,"COSMIC910706,ACH-000245,CVCL_1087,CCLE_Name:BL...","STR Profile: Amelogenin: X,Y; CSF1PO: 10,11 (C...",Yes,https://www.dsmz.de/collection/catalogue/detai...
1867,SIDM01029,KYSE-270,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 15172977,PMID: 1728357,PMID: 25877200,PM...",DSMZ,Commercial,ACC-380,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1350,"COSMIC907319,ACH-000873,CVCL_1350,CCLE_Name:KY...","STR Profile: Amelogenin: X,Y; CSF1PO: 12; D2S1...",Yes,https://www.dsmz.de/collection/catalogue/detai...


In [385]:
supplier_name

'RIKEN'

In [1108]:
def fetch_attribute_cultcol(driver, df):
    for art in driver.find_elements(By.CLASS_NAME, 'pcg-product-info'):
        if art.text.strip() == '':
            continue
        row = BeautifulSoup(art.get_attribute("outerHTML"), 'html.parser')
        h = [r.text.strip() for r in row.find_all('h2')]
        d = [r.text.strip() for r in row.find_all('div')]
        hd = list(zip(h,d))
        for r in hd:
            df[r[0].lower().replace(' ', '_')] = r[1]
    return df

def get_atcc_attributes(first, driver, text, model_id):
    if first:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Allow all cookies']"))).click()
        first = False
    sleep(2)
    try: 
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Expand all')]"))).click()
        headers = list()
        h3 = driver.find_elements(By.XPATH, "//h3")
        append_it = False
        for h in h3:
            if len(headers) == 0:
                headers.append('Product Information')
            if h.text == "General" or h.text == 'Characteristics' or append_it:
                headers.append(h.text)
                append_it = True
            if h.text == "Legal disclaimers":
                append_it = False
        
        dls = driver.find_elements(By.XPATH, "//dl")
        i = 0
        df = pd.DataFrame()
        df['catalog'] = [url.replace('https://www.atcc.org/products/', '').upper()]
        df['model_id'] = model_id
        for dl in dls:
            soup = BeautifulSoup(dl.get_attribute("outerHTML"), 'html.parser')
            dt_elements = soup.find_all('dt', class_='product-information__title')
            dd_elements = soup.find_all('dd', class_='product-information__data')
            dt_values = [headers[i].replace(' ', '_').lower()+'.'+dt.get_text(strip=True).replace(' ', '_').lower() for dt in dt_elements]
            i = i+1
            dd_values = [dd.get_text(strip=True) for dd in dd_elements]
            for j in range(len(dt_values)):
                df[dt_values[j]] = dd_values[j]  
        text = pd.concat([text, df]).reset_index(drop=True)
    except:
        pass
    return first, text

def get_cellbank_riken(driver, text, model_id, model_number):
    tables = driver.find_elements(By.XPATH, "//table")
    main_table = tables[0]
    for table in tables:
        if table.text.startswith('Reference'):
            reference_table = table.text
            break
    main_table = BeautifulSoup(main_table.get_attribute("outerHTML"), 'html.parser')
    rows = main_table.find_all('tr')
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    output = list()
    column_names = ["Depositor", "Originator", "Year of deposit", "Animal", "Genus", "Species", "Race", "Gender", "Age at sampling", "Tissue", "Disease name", "Metastatic ability", "Metastatic tissue", "Classification", "Year of origin", "Lifespan", "Morphology", "Cellosaurus(Expasy)", "Medium", "Culture type", "Culture medium", "Culture method", "Antibiotics", "Passage method", "SC frequency", "Temperature", "CO2 concentration", "Freeze medium", "Freezing method", "Mycoplasma", "Virus (HIV)", "Isozyme", "Chromosome mode", "STR(human)"]
    column_names = [f.lower() for f in column_names]
    for row in rows:
        values = row.find_all('td')
        if len(values) > 0:
            if int(values[0].get('colspan', 1)) > 1:
                continue 
            for val in values:
                val_t = val.text.replace('\t', '').replace('\n', '').lower()
                if (int(val.get('rowspan', 1)) > 1 and val.text != '\n') or val_t == "" or val_t == 'deposit info' or val_t == 'lot info':
                    continue
                if len(output) > 0 and output[-1] in column_names and val_t in column_names:
                    output.append('')
                output.append(val_t)
    for iter in range(0, len(output), 2):
        df[output[iter]] = output[iter+1]
    pattern = r'PubMed ID:\s*(\d+)'
    match = search(pattern, reference_table)
    if match:
        pubmed_id = match.group(1)
        pubmed_id = 'PMID: '+ pubmed_id
    else:
        pubmed_id = ''
    df['publications'] = pubmed_id
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_kclb_attributes(driver, text, model_id, model_number, model_name):
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f"//a[contains(text(), 'SNU-{sub('^0*', '', model_number)}')]")))
        sleep(3)
        element.click()
    except Exception as e:
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f"//a[contains(text(), '{model_name}')]")))
            sleep(3)
            element.click()
        except Exception as e:
            print(f'{model_id}: KCLB:{model_number}: Click error')
            pass
    if len(driver.find_elements(By.CLASS_NAME, "cell_con_tab_wrap")) > 0:
        table = BeautifulSoup(driver.find_elements(By.CLASS_NAME, "cell_con_tab_wrap")[0].get_attribute("outerHTML").replace('\t','').replace('\n',''), 'html.parser')
        headers = table.find('ul').find_all('li')
        headers = [[h['data-ref'], h.text] for h in headers]
        table = table.find_all('div')
        df = pd.DataFrame()
        df['catalog'] = [model_number]
        df['model_id'] = model_id
        tab = list()
        for t in table:
            if "id" in t.attrs.keys():
                tab.append(t)
        table = tab
        for i in range(0, len(table)):
            head = headers[i][0]
            tab = table[i]
            if tab is not None and "id" in tab.attrs.keys() and tab['id'] == head:
                tab = tab.find('table')
                if tab is not None:
                    tab = tab.find_all('tr')
                    if tab is not None:
                        for j in range(len(tab)):
                            if head == 'str' or tab[j].find('th') is None:
                                col = head
                            elif head == 'morphology':
                                continue
                            else:
                                col = head+'.'+ tab[j].find('th').text.replace(' ', '_').lower() 
                            df[col] = tab[j].find('td').text 
            else:
                print(f'{model_id}: KCLB:{model_number}: Missmatch id for table and header {tab.attrs}, {head}')
        text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_iclc_attributes( driver, text, model_id, model_number, model_name):
    tables = driver.find_elements(By.XPATH, "//table")
    table = BeautifulSoup(tables[1].get_attribute("outerHTML"), 'html.parser')
    table = table.find_all('tr')
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    for t in table:
        if t.text.replace('\n', '').__contains__('Close window'):
            continue
        data = t.find_all('td')
        if data[0].text.replace('\xa0', ' ').replace(' ','_').lower().__contains__('bibliography'):
            pattern = r'PMID:\s*(\d+)'
            match = search(pattern, data[1].text.replace('\xa0', ' ').replace('\t',' ').replace('\n',' '))
            if match:
                pubmed_id = match.group(1)
                pubmed_id = 'PMID: '+ pubmed_id
            else:
                pubmed_id = ''
            df['publications'] = pubmed_id
        else:
            df[data[0].text.replace('\xa0', ' ').replace(' ','_').lower()] = data[1].text.replace('\xa0', ' ').replace('\t',' ').replace('\n',' ')
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_dsmz_attributes( driver, text, model_id, model_number):
    divs = driver.find_elements(By.CLASS_NAME, "field")
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    for div in divs:
        soup = BeautifulSoup(div.get_attribute("outerHTML"), 'html.parser')
        label_element = soup.find(class_='label')
        value_element = soup.find(class_='value')
        label = label_element.text.strip().replace(':', '').replace(' ', '_').replace('.', '').lower()
        value = value_element.text.strip()
        if label.__contains__('nbsp') or label.__contains__('Supplied'):
            continue
        df[label] = value
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_addexbio_attributes(driver, text, model_id, model_number):
    divs = driver.find_elements(By.ID, "recommended")
    publications = driver.find_elements(By.ID, "ref")
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    if len(publications)>0:
        df['publications'] = publications[0].text
    string = sub('<font.*blue\">|</font>|<font face="arial">|\n', '', divs[0].get_attribute("outerHTML"))
    string = sub('</p><h2>','$',string)
    string = sub('</h2><p>','',string)
    string = sub('<div id="recommended"><h2>','',string)
    string = sub('</h2> <p>','',string)
    string = string.split('$')
    output = list()
    for s in string:
        if s.startswith('Mutations') or s.startswith('Related cells by') or s.startswith('Citation'):
            continue
        if s.startswith('Sterility'):
            s = s.replace('Sterility:', '').replace(': ', ':').replace(': ', ':').replace('</p><p>', ';')
            s = s.split(';')
            for ss in s:
                output.append("sterility."+ss)
            continue
        if s.startswith('Pathogens'):
            s = s.replace('Pathogens:', '').replace(': ', ':').replace(': ', ':').replace('</p><p>', ';')
            s = s.split(';')
            for ss in s:
                output.append("pathogens."+ss)
            continue
        output.append(s.replace('</p><p>', ';').replace(': ', ':').replace(': ', ':'))
    for s in output:
        out = s.split(':')
        df[out[0].lower()] = out[1]
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_cellbank_jp_attributes(driver, text, model_id, model_number):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'submit')]"))).click()
    except Exception as e:
        pass
    tables = driver.find_elements(By.XPATH, "//table")
    lot = True
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    for t in tables:
        if (t.text.__contains__('Detection of virus genome') or t.text.__contains__('Reference') or t.text.__contains__('Cell Name')) and lot:            
            tab = BeautifulSoup(t.get_attribute("outerHTML"), 'html.parser')
            trs = tab.find_all('tr')
            for tr in trs:
                h = [h.text.strip() for h in tr.find_all('th')]
                d = [d.text.strip() for d in tr.find_all('td')]
                row = list(zip(h,d))
                for r in row:
                    df[r[0]] = r[1]
        if t.text.__contains__('Lot Specification'):
            tab = BeautifulSoup(t.get_attribute("outerHTML"), 'html.parser')
            trs = tab.find_all('tr')
            for tr in trs:
                h = [h.text.strip() for h in tr.find_all('th')]
                d = [d.text.strip() for d in tr.find_all('td')]
                row = list(zip(h,d))
                for r in row:
                    df[r[0]] = r[1]
            lot = False
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_cellbank_australia_attributes(driver, text, model_id, model_number):
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    div = driver.find_elements(By.ID, 'content_description')[0]
    div = BeautifulSoup(div.get_attribute("outerHTML"), 'html.parser')
    df['publications'] = 'PMID: '+div.text.strip().split('PubMed ')[1][0:8].replace('\n', '')
    div = [r for r in div.find_all('p') if r.text.strip() != '']
    div = [r.text.strip().replace("\xa0", "").replace(": ", ":").replace('PubMed ', 'PMID:').replace('Reference:', '').split(":") for r in div if not r.text.__contains__('Ratio') or r.text.__contains__(';')]
    div = [r for r in div if len(r)>1]
    for r in div:
        df[r[0].lower().replace(' ', '_')] = r[1]
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_phe_culture_attributes(driver, text, model_id, model_number):
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Recommended Settings')]"))).click()
    except Exception as e:
        pass
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Characteristics')]"))).click()
        df = fetch_attribute_cultcol(driver, df)
    except Exception as e:
        pass
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Culture Conditions')]"))).click()
        df = fetch_attribute_cultcol(driver, df)
    except Exception as e:
        pass
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Applications')]"))).click()
        df = fetch_attribute_cultcol(driver, df)
    except Exception as e:
        pass    
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_cancertools_attributes(driver, text, model_id, model_number):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[text()='Allow all']"))).click()
    except Exception as e:
        pass
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id   
    div = driver.find_elements(By.CLASS_NAME, 'product-tabs-content')[0]
    div = BeautifulSoup(div.get_attribute("outerHTML"), 'html.parser')
    div = div.find_all('div')[0].find_all('div')
    for d in div:
        if d.text.strip() == '':
            continue
        title = d.find('h4').text.strip().replace(' ', '_').lower()
        rows = d.find_all('li')
        pmid = ''
        for r in rows:
            if title == 'references':
                pattern = r'PMID:\s*(\d+)'
                match = search(pattern, r.find('strong').text.strip())
                if match:
                    pubmed_id = match.group(1)
                    pubmed_id = 'PMID: '+ pubmed_id
                else:
                    pubmed_id = ''
                if pubmed_id != '':
                    pmid = f"{pmid},{pubmed_id}"
                continue
            else:
                col = f"{title}.{r.find('strong').text.strip().replace(':', '').replace(' ', '_').lower()}"            
                entry = f"{r.find('strong').find_next_sibling(text=True).strip()}"
            df[col] = entry
    df['publications'] = pmid[1:]
    text = pd.concat([text, df]).reset_index(drop=True)
    return text

def get_cls_attributes(driver, text, model_id, model_number):
    df = pd.DataFrame()
    df['catalog'] = [model_number]
    df['model_id'] = model_id
    divs = driver.find_elements(By.ID, 'collapseFooter3')
    for div in divs:
        if div.text.strip() == "":
            continue
        table = BeautifulSoup(div.get_attribute("outerHTML"), 'html.parser').find('table')
        if table is None:
            continue
        h = [h.text.strip().replace(' ', '').lower() for h in table.find_all('th')]
        d = [d.text.strip() for d in table.find_all('td')]
        row = list(zip(h,d))
        for r in row:
            df[r[0]] = r[1]
    text = pd.concat([text, df]).reset_index(drop=True)
    return text


In [1126]:
def fetch_in_vitro_attributes(in_vitro_models):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode if desired
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    first = True
    atcc, riken, kclb, dsmz  = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    iclc, addexbio, cellbankaus, cellbankjp  = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    cls, pheuk, cancertools  = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for i in tqdm(range(in_vitro_models.shape[0])):
        url = in_vitro_models['vendor_link'][i]
        model_id = in_vitro_models['model_id'][i]
        model_number = in_vitro_models['catalog_number'][i]
        model_name = in_vitro_models['model_name'][i]
        if (url == 'Not provided' or url.__contains__('bcrj.org.br') or url.__contains__('www.rockland.com') or 
                url.__contains__('www.cctcc.org') or url.__contains__('dtp.cancer.gov') or url.__contains__('ximbio.com') or url.__contains__('www.merckmillipore.com') or url.__contains__('catalog.coriell.org') or url.__contains__('www.fli.de') or url.__contains__('www.ibvr.org') or url.__contains__('catalog.bcrc.firdi.org.tw') or url.__contains__('www.cccells.org') or url.__contains__('www.cellosaurus.org') or url.__contains__('bioivt.com') or url.__contains__('www.kmcellbank.com') or url.__contains__('www.ubigene.us') or url.__contains__('www.abcam.com')):
            continue
        driver.get(url)
        try:
            if url.__contains__('atcc.org'):
                first, atcc = get_atcc_attributes(first, driver, atcc, model_id)
            elif url.__contains__('cellbank.snu.ac.kr'):
                kclb = get_kclb_attributes(driver, kclb, model_id, model_number, model_name)
            elif url.__contains__('cellbank.brc.riken.jp'):
                riken = get_cellbank_riken(driver, riken, model_id, model_number)
            elif url.__contains__('www.iclc.it'):
                iclc = get_iclc_attributes(driver, iclc, model_id, model_number, model_name)
            elif url.__contains__('dsmz.de'):
                dsmz = get_dsmz_attributes(driver, dsmz, model_id, model_number)
            elif url.__contains__('addexbio.com'):
                addexbio = get_addexbio_attributes(driver, addexbio, model_id, model_number)
            elif url.__contains__('cellbank.nibiohn.go.jp'):
                cellbankjp = get_cellbank_jp_attributes(driver, cellbankjp, model_id, model_number)
            elif url.__contains__('www.cellbankaustralia.com'):
                cellbankaus = get_cellbank_australia_attributes(driver, cellbankaus, model_id, model_number)
            elif url.__contains__('www.phe-culturecollections.org.uk'):
                pheuk = get_phe_culture_attributes(driver, pheuk, model_id, model_number)
            elif url.__contains__('www.cancertools.org'):
                cancertools = get_cancertools_attributes(driver, cancertools, model_id, model_number)
            elif url.__contains__('cls.shop'):
                cls = get_cls_attributes(driver, cls, model_id, model_number)
        except Exception as e:
            print('Error')
    driver.quit()
    return [atcc, riken, kclb, dsmz, iclc, addexbio, cellbankaus, cellbankjp, cls, pheuk, cancertools]

In [ ]:
attributes_dfs = fetch_in_vitro_attributes(cmp_sheet)
attributes_dfs

In [1313]:
[atcc, riken, kclb, dsmz, iclc, addexbio, cellbankaus, cellbankjp, cls, pheuk, cancertools] = attributes_dfs
out_cols = ['model_id', 'morphology', 'growth_properties',
       'growth_media', 'morphological_features',
       'validation_technique', 'virology_status', 'other_publications',
       'other_comments', 'contaminated', 'contamination_details',
       'supplements', 'media_id', 'tumour_status', 'model_purity', 'SNP_analysis', 'passage_number']

In [1262]:
riken['growth_properties'] = riken['culture type'].fillna('Not provided').str.replace(' cells', '')
riken['growth_media'] = riken['culture medium'].str.replace(' + ', '+').str.split('+', n=1, expand=True)[0].str.upper()
riken['supplements'] = riken['culture medium'].str.replace(' + ', '+').str.split('+', n=1, expand=True)[1].str.upper().str.replace('+', ' + ').fillna('Not provided')
riken['media_id'] = riken['culture medium'].str.replace(' + ', '+').str.split('+', n=1, expand=True)[0].str.upper().replace({"RPMI1640": "ATCC 30-2001", "MEM": "SIGMA M4655", "DMEM (LOW GLUCOSE)": "GIBCO 11885-084", "HAMF10": "SIGMA N6908", "(HAMF12 : RPMI1640 = 1 : 1)": "SIGMA N6658",
"HAMF12": "SIGMA N6658", "DMEM/HAMF12": "GIBCO 11320-033", "GIT": "Kohjin-bio Co.,Ltd 637-25715", "MCCOY'S 5A": "GIBCO 16600-082", "HUMEDIA-KG2": "KURABO KK-2150S"
})
riken['morphological_features'] = riken['morphology']
#riken['STR_analysis'] = ['Yes' if r != 'No' else r for r in riken['str(human)'].fillna('No')]
riken['validation_technique'] = "PCR"
na_str = "Not provided"
virus = list()
for x, y, z, a, b in zip(
        ["HIV: Not detected" if r == "(-)" else r if r == na_str else f"HIV: {r}" for r in riken['virus (hiv)'].fillna(na_str)],
        ["HCV: Not detected" if r == "(-)" else r if r == na_str else f"HCV: {r}" for r in riken['virus (hcv)'].fillna(na_str)],
        ["HBV: Not detected" if r == "(-)" else r if r == na_str else f"HBV: {r}" for r in riken['virus (hbv)'].fillna(na_str)],
        ["EBV: Not detected" if r == "(-)" else r if r == na_str else f"EBV: {r.replace('(+)', 'Detected')}" for r in riken['virus (ebv)'].fillna(na_str)],
        ["HTLV: Not detected" if r == "(-)" else r if r == na_str else f"HTLV: {r.replace('dna (+)', 'Detected')}" for r in riken['virus (htlv-1)'].fillna(na_str)] ):
    out = ''
    if x != na_str:
        out += x + ','
    if y != na_str:
        out += y + ','
    if z != na_str:
        out += z + ','
    if a != na_str:
        out += a + ','
    if b != na_str:
        out += b + ','
    out = out[:-1]
    if out == '':
        out = na_str
    virus.append(out)
riken['virology_status'] = virus
riken['other_publications'] = riken['publications']
riken['other_comments'] = "Passage method: " + riken['passage method']+ ", Passage ratio: " + riken['passage ratio'] + ", Isozyme: " + riken['isozyme'].str.upper() 
riken['contaminated'] = ["No" if r == "(-)" else r if r == na_str else "Yes" for r in riken['mycoplasma'].fillna(na_str)]
riken['contamination_details'] = ["No contamination" if r == "(-)" else r if r == na_str else "mycoplasma" for r in riken['mycoplasma'].fillna(na_str)]
riken['model_purity'] = riken['metastatic ability'].fillna('no')
riken['tumour_status'] = na_str
riken['SNP_analysis'] = na_str
riken['passage_number'] = na_str
riken = riken[out_cols]

kclb['growth_properties'] = kclb['info.growth_pattern'].fillna(na_str)
kclb['growth_properties'] = ['monolayer' if gp.__contains__('monolayer') else gp for gp in kclb['growth_properties']]
kclb['growth_media'] = kclb['method.kclb_media'].str.replace('  ', ' ').str.replace('Eagle', 'DMEM').str.replace('Dulbecco', 'DMEM').fillna(na_str).split(';', expand=True)[0].fillna(na_str)
kclb['supplements'] = kclb['method.kclb_media'].str.replace('  ', ' ').str.replace('Eagle', 'DMEM').str.replace('Dulbecco', 'DMEM').fillna(na_str).str.split(';', expand=True)[1].fillna(na_str)
kclb['media_id'] = ["ATCC 30-2002" if r.__contains__('DMEM') else "ATCC 30-2001" if r.__contains__('RPMI1640') else na_str for r in kclb['growth_media']]
kclb['morphological_features'] = kclb['info.cellular_morphology'].fillna("Not provided")
kclb['validation_technique'] = "PCR"
kclb['virology_status'] = na_str
kclb['other_comments'] = "Growth rate: " + kclb['info.growth_rate'].fillna(na_str)  + ", Split ratio: " + kclb['method.split_ratio'].fillna(na_str) + ", Doubling time: " + kclb['method.populaton_doubling_time'].fillna(na_str)
kclb['other_publications'] = kclb['references']
kclb['contaminated'] = na_str
kclb['contamination_details'] = na_str
riken['tumour_status'] = na_str
riken['model_purity'] = na_str
riken['SNP_analysis'] = na_str
riken['passage_number'] = na_str
kclb = kclb[out_cols]

dsmz['morphological_features'] = dsmz['morphology'].str.replace('image', '').str.replace('; ', '')
dsmz['growth_properties'] = ['suspension' if gp.__contains__('suspension') else 'adherent' if gp.__contains__('adherent') else 'monolayer' if gp.__contains__('monolayer') else 'Not provided' for gp in dsmz['morphological_features'].str.lower().fillna(na_str)]
dsmz['growth_media'] = dsmz['medium'].str.replace(' + ', '+').str.split('+', n=1, expand=True)[0].fillna(na_str)
dsmz['supplements'] = dsmz['medium'].str.replace(' + ', '+').str.split('+', n=1, expand=True)[1].str.replace('+', ' + ').fillna(na_str)
dsmz['media_id'] = ['DMEM' if med.__contains__('DMEM') else 'RPMI1640' if med.__contains__('RPMI 1640') else "HamF12" if med.__contains__("Ham's F12") else  'MEM' if med.__contains__('MEM') and med != 'DMEM' and not med.__contains__('alpha') else med 
                    for med in dsmz['medium'].str.replace(' + ', '+').str.split('+', n=1, expand=True)[0].str.replace('\d+-?\d*\.?\d+\%\s', '',regex=True).fillna(na_str)]
dsmz['media_id'] = dsmz['media_id'].replace({"McCoy's 5A": "SIGMA M9309", "RPMI1640": "ATCC 30-2001", "DMEM": "ATCC 30-2002", "Iscove's MDM": "SIGMA I3390", "alpha-MEM (with ribo- and deoxyribonucleosides)": "SIGMA M4526", "HamF12": "SIGMA N6658", "MEM": "SIGMA M4655", "Ham's F10": "SIGMA N6908", "NCTC-109": "SIGMA N1140"})
dsmz['validation_technique'] = "PCR assays/ELISA/RNA Hybridization"
dsmz['virology_status'] = dsmz['viruses'].str.replace(' -', ': Not detected').str.replace(' negative', ': Not detected').str.replace('PCR:|ELISA:', '',regex=True).str.replace(' +', ': Detected').str.replace(';', ',').fillna(na_str)
dsmz['other_comment'] = "Doubling time: " + dsmz['doubling_time'] + ", Harvest: " + dsmz['harvest']
dsmz['other_publications'] = ''
dsmz['contaminated'] = 'No'
dsmz['contamination_details'] = dsmz['mycoplasma']
dsmz['model_purity'] = na_str
dsmz['passage_number'] = na_str
dsmz['tumour_status'] = ['Yes' if f.__contains__('expression') else 'No' for f in dsmz['molec_genetics'].str.lower().fillna(na_str)]
dsmz['SNP_analysis'] = ['Yes' if f.__contains__('mutations') else 'No' for f in dsmz['molec_genetics'].str.lower().fillna(na_str)]
dsmz = dsmz[out_cols]

iclc['growth_properties'] = iclc['morphology_and_growth'].str.replace('continuous culture, ', '').str.replace('in|as|grown| ', '',regex=True).str.split(',',n=1,expand=True)[0].fillna(na_str)
iclc['morphological_features'] = iclc['morphology_and_growth'].str.replace('continuous culture, ', '').str.replace('morphology ', '',regex=True).str.replace('neuroblastic-like', 'neuroblast-like').str.split(',',n=1,expand=True)[1].fillna(na_str)
media = {"DMEM": "", "RPMI 1640": "", "Ham's F10": ""}
iclc['growth_media'] = [f if f in media.keys() else na_str for f in iclc['culture_conditions'].str.replace(' + ', '+').str.split('+', n=1, expand=True).fillna(na_str)[0]]
iclc['media_id'] = iclc['growth_media'].replace({"RPMI 1640": "ATCC 30-2001", "DMEM": "ATCC 30-2002", "Ham's F10": "SIGMA N6908"})
iclc['supplements'] = iclc['culture_conditions'].str.replace(' + ', '+').str.split('+', n=1, expand=True).fillna(na_str)[1].str.replace('+', ' + ').str.replace(';.*', '',regex=True)
iclc['validation_technique'] = 'PCR'
iclc['other_comments'] = 'Culture notes: ' + iclc['culture_conditions'].str.split('; ', n=1, expand=True).fillna(na_str)[1]
iclc['other_comments'] = iclc['comments'].fillna('') + iclc['other_comments']
iclc['virology_status'] = na_str
iclc['other_publications'] = iclc['publications']
iclc['contaminated'] = na_str
iclc['contamination_details'] = na_str
iclc['model_purity'] = na_str
iclc['tumour_status'] = na_str
iclc['SNP_analysis'] = na_str
iclc['passage_number'] = iclc['passage_number'].fillna(na_str)
iclc = iclc[out_cols]

addexbio['growth_properties'] = ['Mix of adherent and suspension' if f.__contains__('mixed') else 'Adherent' if f.__contains__('adherent') else 'Suspension' if f.__contains__('suspension') else na_str for f in addexbio['properties'].str.lower().fillna(na_str)]
addexbio['morphological_features'] = addexbio['morphology'].str.replace('(;|\.).*|, epithelial-like', '', regex=True).str.replace(' $', '', regex=True)
addexbio['growth_media'] = addexbio['medium'].str.replace('\s*\(.*', '', regex=True).str.replace(' Medium', '').str.replace(' + ', '+').str.split('+', n=1, expand=True).fillna(na_str)[0].str.replace('^\s|\s$', '', regex=True).str.replace('RPMI1640', 'RPMI-1640').str.replace('AddexBio-Formulated DMEM', 'AddexBio-formulated DMEM').str.replace("Iscove's Modified Dulbecco's", "IMDM")
addexbio['media_id'] = addexbio['growth_media'].replace({"AddexBio-formulated DMEM": "AddexBio C0003-01", "AddexBio-formulated EMEM": "AddexBio C0005-01", "AddexBio-formulated Leibovitz's L-15": "AddexBio C0006-01", "AddexBio-formulated RPMI-1640": "AddexBio C0004-01", "EMEM": "AddexBio C0005-01", "DMEM/F12": na_str, "DMEM/F12": "GIBCO 11320-033", "F12K": "SIGMA N3520", "Ham's F-12": "SIGMA N6658", "IMDM": "SIGMA I3390", "McCoy's 5a": "SIGMA M9309", "RPMI-1640": "AddexBio C0004-01"})
addexbio['supplements'] = addexbio['medium'].str.replace('\s*\(.*', '', regex=True).str.replace(' Medium', '').str.replace(' + ', '+').str.split('+', n=1, expand=True).fillna(na_str)[1].str.replace('^\s|\s$', '', regex=True).str.replace('+', ' + ')
addexbio['validation_technique'] = 'PCR'
addexbio['other_comments'] = ''
addexbio['virology_status'] = ['HIV: resitant' if a or b or c or d or e or f else 'Not detected' for a,b,c,d,e,f in zip([False if f == 'Negative' else True for f in addexbio['pathogens.cmv'].str.replace('\t|</p></div>', '', regex=True).fillna('Negative')],
[False if f == 'Negative' else True for f in addexbio['pathogens.hiv'].str.replace('\t|</p></div>', '', regex=True).fillna('Negative')],
[False if f == 'Negative' else True for f in addexbio['pathogens.hepatitis b'].str.replace('\t|</p></div>', '', regex=True).fillna('Negative')],
[False if f == 'Negative' else True for f in addexbio['pathogens.hepatitis c'].str.replace('\t|</p></div>', '', regex=True).fillna('Negative')],
[False if f == 'Negative' else True for f in addexbio['pathogens.hpv'].str.replace('\t|</p></div>', '', regex=True).fillna('Negative')],
[False if f == 'Negative' else True for f in addexbio['pathogens.ebv'].str.replace('\t|</p></div>', '', regex=True).fillna('Negative')])]
addexbio['other_publications'] = ''

addexbio['contaminated'] = ['Yes' if x or y or z else 'No' for x,y,z in zip([False if f == 'Negative' else True for f in addexbio['sterility.bacteria'].fillna('Negative')],
[False if f == 'Negative' else True for f in addexbio['sterility.yeast'].fillna('Negative')],
[False if f == 'Negative' else True for f in addexbio['sterility.mycoplasma'].fillna('Negative')])]
addexbio['contamination_details'] = [f"{x}, {y}, {z}"for x,y,z in zip(['Bacteria: Not detected' if f == 'Negative' else 'Bacteria: Detected' for f in addexbio['sterility.bacteria'].fillna('Negative')],
['Yeast: Not detected' if f == 'Negative' else 'Yeast: Detected' for f in addexbio['sterility.yeast'].fillna('Negative')],
['Mycoplasma: Not detected' if f == 'Negative' else 'Mycoplasma: Detected' for f in addexbio['sterility.mycoplasma'].fillna('Negative')])]

addexbio['model_purity'] = na_str
addexbio['tumour_status'] = na_str
addexbio['SNP_analysis'] = na_str
addexbio['passage_number'] = na_str
addexbio = addexbio[out_cols]

In [1405]:
cellbankaus

,catalog,model_id,publications,description,description_key_words,also_known_as,organism,strain,tissue,growth_properties,...,references,"int._j_cancer,_60",pmid,other_references,"leonard_j.h.,_ramsay,_j.r.,_kearsley,_j.h._and_birrell,_g.w._radiation_sensitivity_of_merkel_cell_carcinoma_cell_lines._int._j._rad._oncol._biol._phys.,_32","leonard,_j.h,_cook,_a.l,_van_gele,_m,_boyle,_g.m,_inglis,_k.j,_speleman,_f.,_and_sturm,_r.a._proneural_and_proneuroendocrine_transcription_factor_expression_in_cutaneous_mechanoreceptor_(merkel)_cells_and_merkel_cell_carcinoma._int_j_cancer,_2002;_101","k._m._garneski,_a._h._warcola,,_j._h._leonard,_paul_nghiem._merkel_cell_polyomavirus_is_more_frequently_present_in_north_american_than_australian_merkel_cell_carcinoma_tumors._j._invest_dermatol._2009_jan;129(1)",cellosaurus,additional_information,j_natl_cancer_inst._1985_apr;74(4)
0,10092302,SIDM00007,PMID: 7814141,MCC13 is a merkel cell carcinoma cell line; it...,Humanmerkel cell carcinoma,No other names,Human (Homo sapiens),N/A,Skin,Adherent,...,,"100-107, 1995.",7814141,,"1401-1407, 1995.",103B110.,246-8.,CVCL_2583,NaN,NaN
1,10092301,SIDM00014,PMID: 3857372,LIM1215 is a human colorectal carcinoma cell l...,Human colorectal carcinoma,No other names,Human (Homo sapiens),N/A,Omental metastasis,LIM1215 is loosely adherent and grows in islan...,...,NaN,NaN,3857372,NaN,NaN,NaN,NaN,CVCL_2574,"Colorectal cancer from HPNCC patient, mutated ...",759-65
2,10092303,SIDM01360,PMID: 7814141,Human merkel cell carcinoma cell line,Human merkel cell carcinoma,No other names,Human (Homo sapiens),NaN,Skin,Adherent,...,,"100-107, 1995.",7814141,NaN,NaN,NaN,NaN,CVCL_2584,NaN,NaN


In [1406]:
cellbankjp

,catalog,model_id,JCRB No.,Cell Name,Profile,Other Name,Animal,Strain,Genus,Species,...,Pubmed id:4056530,Pubmed id:3782955,Pubmed id:2833349,Pubmed id:2820569,Pubmed id:3731124,Pubmed id:8314315,Pubmed id:8576285,Pubmed id:1922102,Pubmed id:1961183,Pubmed id:6962783
0,JCRB1142,SIDM01612,JCRB1142,HEC-265,"Human endometrioid adenocarcinoma, G1 from ute...",,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JCRB1185,SIDM01658,JCRB1185,KMS-21BM,human myeloma cell line.,,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JCRB0426,SIDM01735,JCRB0426,HuH28,"Human liver, bile duct carcinoma cell line.",,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JCRB1180,SIDM01577,JCRB1180,,human myeloma cell line,,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RCB0466,SIDM00296,JCRB0235,Lu-134-A-H,"small cell carcinoma, classic type",,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,JCRB1022,SIDM00036,JCRB1022,OUMS-23,A human colon carcinoma cell line established ...,,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,Establishment and characterization of a human ...,NaN,NaN,NaN
202,JCRB1020,SIDM01331,JCRB1020,,"Human lung cancer cell line, adenocarcinoma.",,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,JCRB1145,SIDM01614,JCRB1145,HEC-50B,Human endometrioid adenocarcinoma (G3) from ja...,HEC-50,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,JCRB0192,SIDM00568,JCRB0192,,Human gastric cancer cell line from poorly dif...,,human,,Homo,sapiens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Molecular characterization of monoclonal anti-...,[Establishment and characterization of a new g...,NaN


In [1407]:
cls

,catalog,model_id,organism,tissue,disease,metastaticsite,synonyms,description,applications
0,ACC-373,SIDM00028,Human,Lung,Small cell carcinoma,Pericardial effusion,SCLC22H,NaN,NaN
1,RCB1367,SIDM00062,Human,Liver,Hepatoblastoma,NaN,"HUH-6, HuH 6, HuH6, HUH6, Huh6","HuH-6 cells, short for Human Hepatoma 6 cells,...",NaN
2,JCRB0255,SIDM00248,Human,Stomach,Gastric tubular adenocarcinoma,Liver,"MKN74, MKN 74",This cell line was established from tubular ad...,NaN
3,NCI-H3122,SIDM00137,Human,Lung,Adenocarcinoma,NaN,"NCI-H3122, H-3122, NCIH3122",NaN,NaN
4,HTL98011,SIDM00227,Human,Brain,Neuroblastoma,Bone marrow,"Gi-ME-N, Gi-MEN, GI-ME-N, Gimen, Gimen1, Gasli...",GIMEN Cells: Unlocking the Potential of Neurob...,NaN
5,RCB0735,SIDM00239,Human,Kidney,Clear cell renal cell carcinoma,NaN,"OSRC2, RC-2",NaN,NaN
6,RCB0256,SIDM00287,Human,B Lymphocyte,B-cell acute lymphoblastic leukemia,NaN,"Ball-1, Ball 1, BALL1, B-cell Acute Lymphoblas...",This cell line was established in 1976 from th...,NaN
7,RCB0540,SIDM00289,Human,B Cell Precursor Leukemia,NaN,NaN,"HAL01, HAL-1",This cell line was established in 1990 from th...,NaN
8,ACC-72,SIDM00329,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ACC-168,SIDM00346,Human,Blood,Chronic myeloid leukemia,NaN,"LAMA-84, LAMA84, Lama84",The LAMA-84 cell line has been established fro...,NaN


In [1408]:
pheuk

,catalog,model_id,catalogue_no.,cell_line_name,cell_line_description,tissue_of_origin,karyotype,applications,disease,cell_type,subculture_routine,culture_medium,growth_mode,references,morphology,dna_profile_(str_profile),bibliography,receptors,products
0,86022703,SIDM00504,86022703,MOG-G-UVW,Established from an anaplastic astrocytoma of ...,Brain,Not specified,Tumourigenicity and virus studies: HIV,Astrocytoma,Astrocytoma,Split sub-confluent cultures (70-80%) 1:2 to 1...,Ham's F10: DMEM (1:1) + 2mM Glutamine + 10% Fo...,Adherent,"Frame MC, Freshney RI, Vaughan PF, Graham DI, ...",NaN,NaN,NaN,NaN,NaN
1,87061206,SIDM00141,87061206,COLO 685,Established from a 63 year old female with ade...,Uterus,Not specified,Tumourigenicity studies,Adenocarcinoma,Epithelial,"Maintain cultures between 3-9x100,000 cells/ml...",RPMI 1640 + 2mM Glutamine + 10% Foetal Bovine ...,Suspension,Unpublished,Round & Refractile,"Amelogenin: XCSF1PO: 11D5S818: 12,13D7S820: 11...",Not specified,NaN,NaN
2,06030202,SIDM00229,06030202,FTC-236,Derived from a neck lymph node metastasis of a...,Thyroid (lymph node metastasis),NaN,NaN,Carcinoma,Endothelial,Split sub-confluent cultures (70-80%) 1:2 to 1...,DMEM and Ham's F12 (1:1 mixture) + 10ug/ml Ins...,Adherent,"Demeure MJ, Damsky CH, Elfman F, Goretzki PE, ...",NaN,"Amelogenin: XCSF1PO: 10,11D5S818: 11,12D7S820:...","HÃ¶lting T, Zielke A, Siperstein AE, Clark OH,...",NaN,NaN
3,94072237,SIDM00514,94072237,COLO 794,Colo 794 was established from a 14 year old ma...,Skin,Not specified,Cancer studies,Melanoma,Melanocyte,Split sub-confluent cultures (70-80%) 1:2 to 1...,RPMI 1640+ 2mM Glutamine + 10% Foetal Bovine S...,Adherent,Unpublished,Melanocyte,"Amelogenin: X,YCSF1PO: 10D5S818: 11D7S820: 10,...",NaN,NaN,NaN
4,87061209,SIDM00143,87061209,COLO 668,Derived from a brain metastasis of a 47 year o...,Lung (brain metastasis),2n = 46,Tumourigenicity studies,Small Cell Carcinoma,Epithelial,In standard cell culture flasks some cells may...,RPMI 1640 + 2mM Glutamine + 10% Foetal Bovine ...,Suspension and attached,None specified by depositor,Round & Refractile,"Amelogenin: XCSF1PO: 10D5S818: 11D7S820: 8.2,1...","Barretina J, et al., 2012 The Cancer Cell Line...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,94072246,SIDM00916,94072246,RPMI 7932,Derived from the pleural effusion of a 36 year...,Pleural effusion,Not specified,Tumourigenicity studies,Melanoma,Epithelial-like,Split sub-confluent cultures (70-80%) 1:2 to 1...,RPMI 1640 + 2mM Glutamine + 10% Foetal Bovine ...,Adherent,None specified by depositor,NaN,NaN,NaN,NaN,Plasminogen activator
97,08090508,SIDM01406,08090508,UM-UC-13,UM-UC-13 was produced from a lymphatic metasta...,Lymph node,NaN,Cell model for research into bladder cancer,Carcinoma,Epithelial,Split sub-confluent cultures (70-80%) 1:3 to 1...,EMEM (EBSS) + 2mM Glutamine + 1% Non Essential...,Adherent,"Sabichi et al., 2006, Characterization of a pa...",NaN,"Amelogenin: X,YCSF1PO: 10,12D5S818: 9,13D7S820...",NaN,NaN,NaN
98,09100108,SIDM01906,09100108,Mero-95,Mero-95 was derived from a pleural effusion ...,Pleural cavity,Number of chromosomes 54-58,Investigation of the properties of malignant m...,Mesothelioma,Epithelial,Split sub-confluent cultures (70-80%) 1:4 to 1...,Hams F10 + 15% FCS + 2mM Glutamine,Adherent,"Versnel et al., 1989 Establishment of human ma...",NaN,"Amelogenin: X,YCSF1PO: 10,12D5S818: 11,14D7S82...",NaN,NaN,Epithelial membrane antigen (EMA)
99,RCB0480,SIDM00244,99072802,NB69,Sample was isolated from a 16 month old male ...,Brain,Not specified,NaN,Neuroblastoma,NaN,Cells can grow in aggregates as well as attach...,RPMI 1640 + 2mM Glutamine + 15% Foetal Bovine ...,Semi-adherent,None specified by depositor,NaN,"Amelogenin: X,YCSF1PO: 11,12D5S818: 11,12D7S82...",NaN,NaN,NaN


In [1409]:
cancertools

,catalog,model_id,tool_details.name,tool_details.organism,tool_details.tissue,tool_details.disease,tool_details.growth_properties,tool_details.model,tool_details.conditional,tool_details.description,...,handling.volume,handling.temperature,handling.storage_conditions,handling.subculture_routine,handling.mycoplasma_free,tool_details.production_details,tool_details.tool_sub_type,tool_details.parental_cell,target_details.target,handling.atmosphere
0,04072107,SIDM00078,BICR 31 Cell Line,Human,Tongue,Cancer,Adherent,Tumour line,Yes,Adherent cell line derived from a squamous cel...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04072103,SIDM00077,BICR 10 Cell Line,Human,Buccal mucosa,Cancer,Adherent,Tumour line,NaN,Adherent keratinocyte cell line derived from a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10032310,SIDM00470,PEO6 Cell Line,Human,Ovary,Cancer,Adherent,Tumour line,Yes,The PEO6 cell line is an adherent cell line de...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10032309,SIDM00471,PEO4 Cell Line,Human,Ovary,Cancer,Adherent,Tumour line,Yes,The PEO4 cell line is one of nine from the PE ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04072106,SIDM00485,BICR 22 Cell Line,Human,Tongue,Cancer,Adherent,Tumour line,Yes,Adherent cell line derived from a lymph node m...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10032308,SIDM00472,PEO1 Cell Line,Human,Ovary,Cancer,Adherent,Tumourigenic,Yes,The PEO1 Cell Line is one of nine from the PE ...,...,1 ml,37° C,Liquid Nitrogen,Split sub-confluent cultures (70-80%) 1:4 to 1...,Yes,NaN,NaN,NaN,NaN,NaN
6,04072111,SIDM00501,BICR 78 Cell Line,Human,Alveolus,Cancer,Adherent,Tumour line,NaN,Adherent keratinocyte cell line derived from a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,94122104,SIDM00540,DOK Cell Line,Human,Tongue,Cancer,Adherent,Tumour line,Yes,The degree of dysplasia in the patient was des...,...,NaN,NaN,NaN,NaN,NaN,The dysplastic oral keratinocyte cell line DOK...,NaN,NaN,NaN,NaN
8,85061106,SIDM01202,RT112/84 Cell Line,Human,Bladder,Cancer,Adherent,Tumour line,Yes,A slow growing tumour which does not form colo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,93112520,SIDM01211,A2780ADR Cell Line,Human,Ovary,Cancer,Adherent,Tumour line,Yes,The adriamycin-resistant cell line A2780ADR ha...,...,NaN,NaN,NaN,NaN,NaN,Split sub-confluent cultures (70-80%) 1:5 to 1...,Continuous,A2780,Adriamycin resistance,NaN


In [1129]:
atcc 

,catalog,model_id,product_information.product_category,product_information.organism,product_information.cell_type,product_information.morphology,product_information.tissue,product_information.disease,product_information.applications,product_information.product_format,...,handling_information.seeding_density,handling_information.culture_maintenance,history.patent_number,characteristics.passage_number,characteristics.virus_susceptibility,characteristics.immortalization_method,quality_control_specifications.bacterial_and_fungal_testing,characteristics.cells_per_vial,quality_control_specifications.viability,history.chain_of_custody
0,HTTPS://CLS.SHOP/300490,SIDM01229,Human cells,"Homo sapiens, human",neuroblast,neuroblast,Brain,Neuroblastoma,3D cell cultureHigh-throughput screeningImmuno...,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HTTPS://CLS.SHOP/300490,SIDM00019,Human cells,"Homo sapiens, human",NaN,epithelial,Lung,Papillary Adenocarcinoma,3D cell culture,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HTTPS://CLS.SHOP/300490,SIDM01168,Human cells,"Homo sapiens, human",NaN,epithelial,Ovary,Adenocarcinoma,3D cell culture,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HTTPS://CLS.SHOP/300490,SIDM01183,Human cells,"Homo sapiens, human",NaN,epithelial,Lung,Carcinoid,3D cell culture,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HTTPS://CLS.SHOP/300490,SIDM00849,Human cells,"Homo sapiens, human",NaN,epithelial,kidney,Adenocarcinoma,3D cell culture,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,HTTPS://CLS.SHOP/300490,SIDM00404,Human cells,"Homo sapiens, human",NaN,lymphoblast-like,Lymph node,Large Cell Lymphoma,3D cell cultureImmunology,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,HTTPS://CLS.SHOP/300490,SIDM00836,Human cells,"Homo sapiens, human",NaN,epithelial,Large intestine; Colon,"Adenocarcinoma; Colorectal; Dukes' type C, gra...",3D cell culture,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,HTTPS://CLS.SHOP/300490,SIDM00737,Human cells,"Homo sapiens, human",neuroendocrine cell,NaN,Lung,Carcinoma; Non-small cell lung cancer; Stage 4,3D cell culture,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,HTTPS://CLS.SHOP/300490,SIDM00096,Human cells,"Homo sapiens, human",lymphoblast,lymphoblast,Pleural effusion,Large Cell Immunoblastic Lymphoma,3D cell cultureImmunology,Frozen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1123]:
merged = text.merge(cmp_sheet[['model_id', 'vendor_link']], on='model_id', how='left')
merged['vendor_link'].str.split('://', expand=True)[1].str.split('/',expand=True)[0].unique()

array(['cellbank.brc.riken.jp', 'cellbank.snu.ac.kr', 'www.iclc.it',
       'www.dsmz.de', 'www.addexbio.com', 'cellbank.nibiohn.go.jp',
       'www.atcc.org', 'www.phe-culturecollections.org.uk', 'cls.shop',
       'www.cancertools.org', 'www.cellbankaustralia.com'], dtype=object)

In [1124]:
merged[merged['vendor_link'].str.contains('cellbank.brc.riken.jp')]

,catalog,model_id,depositor,originator,year of deposit,original cell,animal,genus,species,tissue,...,Pubmed id:2833349,Pubmed id:2820569,Pubmed id:3731124,Pubmed id:8314315,Pubmed id:8576285,Pubmed id:1922102,Pubmed id:1961183,Pubmed id:6962783,history.chain_of_custody,vendor_link
0,RCB1901,SIDM01774,"obinata, masuo","kobari, masao",2004,tkg0492,_human < mammals,homo,sapiens,pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
32,RCB0443,SIDM00234,"ichimura, hiroshi","ichimura, hiroshi",1989,NaN,_human < mammals,homo,sapiens,cervix,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
41,RCB0444,SIDM00307,"akiyama, mitoshi","akiyama, mitoshi",1989,NaN,_human < mammals,homo,sapiens,lung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
58,RCB1129,SIDM00252,"todoroki, t. & ohno, t.","todoroki, t. & ohno, t. & kim, b.",1994,NaN,_human < mammals,homo,sapiens,"gallbladder, lymph node meta",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
87,RCB1062,SIDM01368,"okabe, tetsuro","okabe, tetsuro",1994,NaN,_human < mammals,homo,sapiens,"stomach, lymph node meta",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,RCB1182,SIDM01259,"tsuchiya, hiroyuki",soheir s. el-sonbaty,1995,tanoue,_human < mammals,homo,sapiens,blood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
1202,RCB1200,SIDM01242,"katayama, hiroshi","katayama, hiroshi",1996,NaN,_human < mammals,homo,sapiens,"melanoma, ascites meta",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
1210,RCB1179,SIDM01753,"imanishi, hiroyasu","imanishi, hiroyasu",1995,NaN,_human < mammals,homo,sapiens,"stomach, ascites meta",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...
1262,RCB0469,SIDM00293,"terasaki, takeo","tsumuraya, m. & nakajima, t.",1990,NaN,_human < mammals,homo,sapiens,lung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cellbank.brc.riken.jp/cell_bank/CellInf...


In [1161]:
cmp_sheet

,model_id,model_name,model_name_aliases,type,parent_id,origin_patient_sample_id,growth_properties,media_id,growth_media,plate_coating,...,publications,supplier,supplier_type,catalog_number,vendor_link,rrid,external_ids,comments,STR_analysis,cell_line_provider_url
0,SIDM01774,PK-59,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 25877200,PMID: 22460905,P...",RIKEN,Commercial,RCB1901,http://cellbank.brc.riken.jp/cell_bank/CellInf...,CVCL_4897,"ACH-000205,CVCL_4897,CCLE_Name:PK59_PANCREAS",STR Profile: Amelogenin: X; CSF1PO: 10; D3S135...,Yes,http://cellbank.brc.riken.jp/cell_bank/CellInf...
1,SIDM00192,SNU-1033,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 10674020,PMID: 19956504,P...",KCLB,Commercial,01033,https://cellbank.snu.ac.kr/main/tmpl/sub_main....,CVCL_5002,"ACH-000286,CVCL_5002,CCLE_Name:SNU1033_LARGE_I...",STR Profile: Amelogenin: X; CSF1PO: 10; D1S165...,Yes,https://cellbank.snu.ac.kr/main/tmpl/sub_main....
2,SIDM01447,SNU-466,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 22460905,PMID: 19956504,P...",KCLB,Commercial,00466,https://cellbank.snu.ac.kr/main/tmpl/sub_main....,CVCL_5064,"ACH-000289,CVCL_5064,CCLE_Name:SNU466_CENTRAL_...","STR Profile: Amelogenin: X,Y; CSF1PO: 12; D3S1...",Yes,https://cellbank.snu.ac.kr/main/tmpl/sub_main....
3,SIDM01554,IST-MES-2,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 22460905,PMID: 10232421,P...",ICLC,Commercial,HTL01007,http://www.iclc.it/details/det_list.php?line_i...,CVCL_1312,"ACH-000331,CVCL_1312,CCLE_Name:ISTMES2_PLEURA","STR Profile: Amelogenin: X,Y; CSF1PO: 10,11; D...",Yes,http://www.iclc.it/details/det_list.php?line_i...
4,SIDM01689,MUTZ-5,Not provided,Cell Line,NaN,NaN,Suspension,Not provided,Not provided,Not provided,...,"PMID: 26589293,PMID: 22460905,PMID: 30285677,P...",DSMZ,Commercial,ACC-490,https://www.dsmz.de/collection/catalogue/detai...,CVCL_1873,"ACH-000492,CVCL_1873,CCLE_Name:MUTZ5_HAEMATOPO...","STR Profile: Amelogenin: X; CSF1PO: 8,9; D2S13...",Yes,https://www.dsmz.de/collection/catalogue/detai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,SIDM02153,MM485,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,PMID: 3127479,ECACC,Commercial,11072803,Not provided,Not provided,"COSMIC1206009,ACH-001973",NaN,,NaN
1928,SIDM02155,NZM3,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,NaN,ECACC,Commercial,17020204,Not provided,Not provided,"COSMIC1660299,ACH-001982",NaN,,NaN
1929,SIDM02157,WM3772F,Not provided,Cell Line,NaN,NaN,Adherent,Not provided,Not provided,Not provided,...,NaN,Rockland,Commercial,WM3772F-01-0001,Not provided,Not provided,ACH-002687,NaN,,NaN
1930,SIDM02159,HAP1,Not provided,Cell Line,SIDM02121,SIDS02033,Adherent,Not provided,Not provided,Not provided,...,NaN,Horizon Discovery,Commercial,C631,Not provided,Not provided,ACH-002475,HAP1 is a subclone of KBM7 which is haploid ex...,,NaN
